In [2]:
from mod1 import *


    
depress('day')

In [2]:
from mod1 import *

def kkk(market='kospi', from_day = '2020-01-01',period ='month'):
    if market=='kospi' or market=='kosdaq':
        df = select_market_period(market,from_day)
    else :
        df = select_stock_period(market,from_day)
        
    df['Date']=pd.to_datetime(df['Date'])
    months = [g for n, g in df.groupby(pd.Grouper(key='Date',freq='M'))]  ##   월별
    weeks = [g for n, g in df.groupby(pd.Grouper(key='Date',freq='W'))]  ##   주별
    columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
    rows = []    

    if period == 'day':
        nick = 'day'
        df=df[['Date','Open', 'High', 'Low','Close', 'Volume']]
        df.columns=columns
        #df = df.set_index(df['date'])
        #return df    
    if period == 'month':
        nick='month'
        period = months
    elif period == 'week':
        nick='week'
        period = weeks
        
    for i in range(len(period)):
        try:
            rows.append(period[i].iloc[-1]['Date'])
            rows.append(period[i].iloc[0]["Open"])
            rows.append(max(period[i]['High']))
            rows.append(min(period[i]['Low']))
            rows.append(period[i].iloc[-1]['Close'])
            rows.append(sum(period[i]['Volume']))
        except:
            pass
    
    print('count:', len(period))
    arr = np.array(rows)
    if nick == 'day':
        return df
    
    if nick == 'week' and len(period) > 177:
        #arr1 = arr.reshape(len(period)-1,6)
        arr1 = arr.reshape(len(period)-1,6)
        
    else:
        arr1 = arr.reshape(len(period),6)
        
    df = pd.DataFrame(data=arr1, columns=columns)
    
    return df

df = kkk(market='kospi', from_day = '2020-01-01', period='week' )
df

count: 62


,Date,Open,High,Low,Close,Volume
0,2020-01-03,2201,2203,2165,2176,1126241000
1,2020-01-10,2155,2207,2138,2206,3261879000
2,2020-01-17,2204,2269,2204,2251,3459492000
3,2020-01-23,2261,2277,2235,2246,2433930000
4,2020-01-31,2192,2200,2119,2119,2814526000
...,...,...,...,...,...,...
57,2021-02-05,2976.81,3142.26,2947.24,3120.13,5317878416
58,2021-02-10,3113.63,3129.66,3064.25,3100.58,5500826524
59,2021-02-19,3108.7,3180.94,3040.28,3107.62,10510590793
60,2021-02-26,3114.03,3142.48,2988.28,3013.69,8110433485


# 변동성이 발생한 기간에서에서 당일까지  종목별 변동내역을 순서대로 출력

In [ ]:
### 특정기간 (ex: 급격한 하락, 상승등) 동안 최저가와   현재 가격을 비교  sorting  _2 


from  mod1 import *

def period_down(from_day, to_day):
    start = time.time()

    df_date = pd.read_sql("select Date from market where Name='hrs' order by Date desc limit 1",engine)
    df_date = pd.to_datetime(df_date['Date'])
    #df = df + timedelta(1)          ##  최종날짜 다음날짜
    df_date = str(df_date)
    standard = df_date[4:14]                ## 2020-07-13

    #query = "select Name, min(Close) from market where Name = '동방' and Date > '2020-12-01' and Date < '2020-12-31' group by Name"
    query1 = "select Name, min(Close) from market where Date >" 
    query2 = "and Date <=" 
    query = query1 +"'"+ from_day +"'"+ query2 +"'"+ to_day +"'"+ " group by Name"
    
    df = pd.read_sql(query, engine)

    df1 = all_stock_at(standard)
    df1_last = df1[['Name','Close']]

    df2 = pd.merge(df, df1_last, on="Name")
    df2['diff']=df2['Close']/df2['min(Close)']
    df2 = df2.sort_values(by=['diff'], ascending='True')
    df2 = df2.reset_index(drop=True)
    #display(df2)
    df2.to_excel(path_down+standard+'.xlsx')
    try:
        os.mkdir(path_down+from_day+'_'+to_day+'/'+standard+'/')
    except:
            pass
    for filename in glob.glob(os.path.join(path_down , '*.*')):
        shutil.copy(filename, path_down+from_day+'_'+to_day+'/'+standard+'/')
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
period_down('2020-02-01', '2020-03-31')

In [ ]:
### 특정기간 (ex: 급격한 하락, 상승등) 동안 최저가화    현재 가격을 비교  일별로  sorting  파일생성


from  mod1 import *

def period_down(from_day, to_day, some_day):  ##  from _day에서 to_day까지 최저가와  some_day부터 당일까지 일별로 가격비교 
    start = time.time()
    
    df = select_stock_period('삼성전자', some_day)
    day_list=[]
    for i in range(df.shape[0]):
        day_list.append(str(df['Date'][i]))
  
    #query = "select Name, min(Close) from market where Name = '동방' and Date > '2020-12-01' and Date < '2020-12-31' group by Name"
    query1 = "select Name, min(Close) from market where Date >" 
    query2 = "and Date <=" 
    query = query1 +"'"+ from_day +"'"+ query2 +"'"+ to_day +"'"+ " group by Name"
    
    df = pd.read_sql(query, engine)
    
    for i in day_list:

        df1 = all_stock_at(i)
        df1_last = df1[['Name','Close']]

        df2 = pd.merge(df, df1_last, on="Name")
        df2['diff']=df2['Close']/df2['min(Close)']
        df2 = df2.sort_values(by=['diff'], ascending='True')
        df2 = df2.reset_index(drop=True)
        #display(df2)
        df2.to_excel(path_down+i+'.xlsx')
    try:
        os.mkdir(path_down+from_day+'_'+to_day+'/')
    except:
            pass
    for filename in glob.glob(os.path.join(path_down , '*.*')):
        try:
            shutil.copy(filename, path_down+from_day+'_'+to_day+'/')
        except:
            pass
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
period_down('2020-02-01', '2020-03-31', '2020-04-01')

In [ ]:
##  period_down 엑셀화일에서 종목을 읽어  그당시의 상태와 현재상태 종가와 비교 (추세 확인)

from mod1 import *

def period_down_analysis(some_day):
    start=time.time()

    query = 'select Date from market order by Date desc limit 1'
    last_day = pd.read_sql(query ,engine)
    last = str(last_day)
    last = last[17:27]

    df = select_stock_period('삼성전자', some_day)
    day_list=[]
    for i in range(df.shape[0]):
        day_list.append(str(df['Date'][i]))

    for i in day_list:
        df_a = pd.read_excel(path_down+i+'.xlsx', index_col=0).reset_index(drop=True)

        df_b = all_stock_at(last)
        df_c = pd.merge(df_a, df_b, on='Name')
        df_c = df_c[['Name', 'Close_x','Close_y']]
        df_c.columns = ['name','closed','close']
        df_c['diff']=df_c['close']/df_c['closed']    

        df_d = df_c.sort_values(by=['diff'], axis=0, ascending=False)
        df_d.to_excel(path_down+'analysis'+'/'+"absolute_"+i+'.xlsx')                          ##  단순비교 순서대로

        df_d = df_d.reset_index()
        df_d = df_d.reset_index()
        df_d['diff_a']=df_d['index']-df_d['level_0']
        df_d=df_d[['index', 'name', 'closed', 'close', 'diff', 'diff_a']]
        df_d=df_d.set_index('index', drop=True)
        df_d = df_d.sort_values(by=['diff_a'], axis=0, ascending=False)
        df_d.to_excel(path_down+'analysis'+'/'+'relative_'+i+'.xlsx')                        ##  당시 순위와  현재순위  상대 비교 
    print("경과시간: ",time.time()-start)
    
period_down_analysis('2021-01-01')

In [ ]:
##  period_down 디렉토리 엑셀화일에서 종목을 읽어  그당시의 상태와 현재상태 종가와 비교

from mod1 import *

def period_down_analysis(some_day):
    start=time.time()

    query = 'select Date from market order by Date desc limit 1'
    last_day = pd.read_sql(query ,engine)
    last = str(last_day)
    last = last[17:27]

    df = select_stock_period('삼성전자', some_day)
    day_list=[]
    for i in range(df.shape[0]):
        day_list.append(str(df['Date'][i]))

    for i in day_list:
        df_a = pd.read_excel(path_down+i+'.xlsx', index_col=0).reset_index(drop=True)

        df_b = all_stock_at(last)
        df_c = pd.merge(df_a, df_b, on='Name')
        df_c = df_c[['Name', 'Close_x','Close_y']]
        df_c.columns = ['name','closed','close']
        df_c['diff']=df_c['closed']/df_c['close']    

        df_d = df_c.sort_values(by=['diff'], axis=0, ascending=False)
        df_d.to_excel(path_down+'analysis'+'/'+"absolute_"+i+'.xlsx')                          ##  단순비교 순서대로

        df_d = df_d.reset_index()
        df_d = df_d.reset_index()
        df_d['diff_a']=df_d['index']-df_d['level_0']
        df_d=df_d[['index', 'name', 'closed', 'close', 'diff', 'diff_a']]
        df_d=df_d.set_index('index', drop=True)
        df_d = df_d.sort_values(by=['diff_a'], axis=0, ascending=False)
        df_d.to_excel(path_down+'analysis'+'/'+'relative_'+i+'.xlsx')                        ##  당시 순위와  현재순위  상대 비교 
    print("경과시간: ",time.time()-start)
    
period_down_analysis('2021-01-01')

In [ ]:
###  vote-stock 을  some_day부터  일별로 생성

from mod1 import *

def vote_stock( some_day ):
    start = time.time()

    df_day = select_stock_period('삼성전자', some_day)
    day_list=[]
    for i in range(df_day.shape[0]):
        day_list.append(str(df_day['Date'][i]))

    df  =all_stock_period('2019-01-01')
    df = df[df['Volume'] >  500000]
    df.reset_index(drop=True)
    #display(df)    

    for i in range(df_day.shape[0]-1):
        from_day=  day_list[i]
        print(from_day)
        to_day = day_list[i+1]
        print(to_day)

        df1 = df[df['Date'].astype(str) == from_day]
        df1 = df1[['Name','Volume','Close']]
        df1.columns = ['Name','yester_Volume','yester_Close']    

        df2 = df[df['Date'].astype(str) == to_day]
        df2 = df2[['Name','Volume','Close']]
        df2.columns = ['Name','today_Volume','today_Close']

        df3 = pd.merge(df1,df2,on='Name')
        df3['Close']=df3['today_Close']/df3['yester_Close']
        df3['Volume']=df3['today_Volume']/df3['yester_Volume']
        df3 = df3.sort_values(by=['Volume','Close'],ascending=False)
        df4 = df3.sort_values(by=['Close','Volume'],ascending=False)
        df3 = df3.reset_index(drop=True)
        df4 = df4.reset_index(drop=True)
        df3 = df3[:15]
        df4 = df4[:15]
        df3.to_excel(path_volume+to_day+'.xlsx', encoding='utf-8')
        df4.to_excel(path_price+to_day+'.xlsx', encoding='utf-8')        
        #display(df3)
        #display(df4)

    print('time:', time.time()-start)
    
vote_stock('2019-01-01' )    

In [ ]:
##  vote_stock 디렉토리 엑셀화일에서 종목을 읽어  그당시의 상태와 현재상태 종가와 비교

from mod1 import *

def vote_stock_analysis(part , some_day):
    start=time.time()
    
    if part ==  "price":
        part = path_price
        filename='price'
    else:
        part = path_volume
        filename = 'volume'
    #print(part)
    
    start=time.time()

    query = 'select Date from market order by Date desc limit 1'
    last_day = pd.read_sql(query ,engine)
    last = str(last_day)
    last = last[17:27]

    df = select_stock_period('삼성전자', some_day)
    day_list=[]
    for i in range(df.shape[0]):
        day_list.append(str(df['Date'][i]))

    for i in day_list:
        df_a = pd.read_excel(part+i+'.xlsx', index_col=0).reset_index(drop=True)
        df_a = df_a[['Name','today_Close']]
        df_a.columns = ['Name','Close']
        df_b = all_stock_at(last)
        df_c = pd.merge(df_a, df_b, on='Name')
        df_c = df_c[['Name', 'Close_x','Close_y']]
        df_c.columns = ['name','closed','close']
        df_c['diff']=df_c['close']/df_c['closed']    

        df_d = df_c.sort_values(by=['diff'], axis=0, ascending=True)
        df_d.to_excel(path_vote_stock+'analysis'+'/'+filename+'_'+"absolute_"+i+'.xlsx')

        df_d = df_d.reset_index()
        df_d = df_d.reset_index()
        df_d['diff_a']=df_d['index']-df_d['level_0']
        df_d=df_d[['index', 'name', 'closed', 'close', 'diff', 'diff_a']]
        df_d=df_d.set_index('index', drop=True)
        df_d = df_d.sort_values(by=['diff_a'], axis=0, ascending=True)
        df_d.to_excel(path_vote_stock+'analysis'+'/'+filename+'_'+"relative_"+i+'.xlsx')
    print("경과시간: ",time.time()-start)
    

list = ['price','volume']
for i in list:
    vote_stock_analysis(i, '2019-01-03')

# depress() 에러날때 오류 발견

In [ ]:
from mod1 import *

df = all_stock_at('2020-06-12')
df = df['Name']
name = df.to_list()
start_day='2020-01-01'
period = 'month'
#period = 'week'
#period = 'day'


for i in name:
    df = day_week_month_data(market=i,start_day=start_day,period=period)
    print(i)
    print(df.iloc[:1])    

# 상장목록 내역 갱신

In [ ]:
##  월별 상장주식종목 갱신

from mod1 import *
from pykrx.stock.api import *
from pykrx import *

start = time.time()  # 시작 시간 저장

date_query = "select Date from market where Name='hrs' order by Date desc limit 1"

df = pd.read_sql(date_query, engine)
df = pd.to_datetime(df['Date'])
df = str(df)
last_date = df[4:14]
last_day = last_date.replace('-','')   ## 20200713
print("최종일자 : ", last_date)

pre_query ="select * from market where Date ="
query = pre_query+'\''+last_date+'\''

#df = get_market_price_change_by_ticker("20200917", "20200918")
#df.to_excel('f:/kkang.xlsx')
a = pd.read_excel('d:/market.xlsx')
df = stock.get_market_ohlcv_by_ticker(last_day, "KOSPI")
df = df.reset_index()
df_kospi=df[['티커','종목명']]

df = stock.get_market_ohlcv_by_ticker(last_day, "KOSDAQ")
df = df.reset_index()
df_kosdaq=df[['티커','종목명']]
b=pd.concat([df_kospi, df_kosdaq])
c = pd.read_sql(query, engine)

a = a.rename(columns={'종목코드':'code','종목명':'name'})
b = b.rename(columns={'티커':'code','종목명':'name'})
c = c.rename(columns={'Code':'code','Name':'name'})
#total_list = list(set(file_list) - set(filter_list))  ## total*.xlsx - total_filter*.xlsx
print(a.shape)  ##  현재 기준이되는 종목수 - market.xlsx
print(b.shape)  ##  거래소 상장 종목수     ( b - a = 10)   우선주 18, 외국상장종목 8   
print(c.shape)  ##  현재 DB에 있는 종목수 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
## 일별 관리종목 추출

from mod1 import *

today = datetime.now()
today = today.strftime("%Y-%m-%d")
#today=input('입력')
#url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page=1'
url = 'https://finance.naver.com/sise/management.nhn'
source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
data = []

path = 'd:\\stockdata\\관리종목\\'+today+'.xlsx'
body = source.find('body')
trs = body.find_all('tr')
name = []
for tr in trs:
    tds = tr.find_all('a',{'class':"tltle"})
    for td in tds:
        name.append(td.text.strip())

df = pd.DataFrame(name)
df['Date']=str(today)
df = df.set_index('Date')
df.columns=['Name']
df.to_excel(path)
df = pd.read_excel(path)
df.to_sql(name='badstock', con=engine, if_exists='append', index = False)

print(df)

# 하위 디렉토리포함하여 DataFrame에 Columns Name변경

In [ ]:
## 하위디렉토리 포함하여 DataFrame Columns명 변경하여 엑셀로 저장

from mod1 import *

path = 'f:/stockdata/close_ma120/2020/'

dir = os.listdir(path)
#print(dir)
for i in dir:
    sdir = os.listdir(path+i)
    for j in sdir:
        ssdir = os.listdir(path+i+'/'+j)
        for k in ssdir:
            files = glob.glob(path+i+'/'+j+'/'+k+'/'+'total_filter_*.*')
            for m in files:
                df = pd.read_excel(m, index_col=0)  ##  unnamed:0 column 생성방지 
                df = df.rename(columns={'name_x':'Name','price_x':'Closed'})
                df.to_excel(m, index=False)

# 기간별 가격추이

In [ ]:
##  날짜별 관심종목 상승평균  
from mod1 import *
df = pd.read_excel('d:/stockdata/vote_stock/analysis/pprice_2019-05-03.xlsx')
df['diff'].mean()

In [ ]:
###  전날대비 가격 상승순,  거래량 증가 순으로 종목 순서대로 정렬

from mod1 import *

start = time.time()

kpi200_df = pd.read_sql("select Date from market where Name='hrs' order by Date desc limit 2", engine)
yesterday = str(kpi200_df['Date'][1])
today = str(kpi200_df['Date'][0])

df  =all_stock_period('2021-02-18')
df = df[df['Volume'] >  500000]
df.reset_index(drop=True)
#display(df)


df1 = df[df['Date'].astype(str) == yesterday]
df1 = df1[['Name','Volume','Close']]
df1.columns = ['Name','yester_Volume','yester_Close']
#display(df1)


df2 = df[df['Date'].astype(str) == today]
df2 = df2[['Name','Volume','Close']]
df2.columns = ['Name','today_Volume','today_Close']
#display(df2)

df3 = pd.merge(df1,df2,on='Name')
df3['Close']=df3['today_Close']/df3['yester_Close']
df3['Volume']=df3['today_Volume']/df3['yester_Volume']
df3 = df3.sort_values(by=['Volume','Close'],ascending=False)
df4 = df3.sort_values(by=['Close','Volume'],ascending=False)
df3 = df3.reset_index(drop=True)

df3 = df3[:15]
df4 = df4.reset_index(drop=True)
df4 = df4[:15]
#df3.to_excel(path_volume+today+'.xlsx', encoding='utf-8')
#df4.to_excel(path_price+today+'.xlsx', encoding='utf-8')        
display(df3)
display(df4)

print('time:', time.time()-start)

In [ ]:
## vote_stock 디렉토리 추이 분석 (함수로 변경_prioce, volume , 기간 선택 , 최종날짜를 DB에서 자동추출)_ 4

from mod1 import *
    
def vote_stock(select):
    start = time.time()
    date_query = "select Date from market where Name='hrs' order by Date desc limit 1"

    df = pd.read_sql(date_query, engine)
    df = pd.to_datetime(df['Date'])
    df = str(df)
    last_date = df[4:14]                ## table에서 최종 날짜 추줄
    
    if select == 'pprice':
        file = glob.glob('d:\\stockdata\\vote_stock\\detect_stock_with_pprice_'+'*.xlsx')
        
    elif select == 'volume':
        file = glob.glob('d:\\stockdata\\vote_stock\\detect_stock_with_volume_'+'*.xlsx')
        
    else:
        print('select error')
        exit()    
    
    
    
    start = time.time()
    
    file = glob.glob('d:\\stockdata\\vote_stock\\'+'*.xlsx')
        
     
    for i in file:
        df = pd.read_excel(i, index_col=0)
        df = df[['Name','Closed']]
        #df = df.rename(columns={'today_Close':'Closed'})
        name = df['Name'].tolist()
        select_query = "select Name,Close from market where Name in ("
        var = ") and  Date = "

        query = select_query+str(name)[1:-1]+var+"'"+last_date+"'"
        df2 = pd.read_sql(query, engine)

        df1 = pd.merge(df, df2)
        df1['diff']=df1['Close']/df1['Closed']
        df1 = df1.sort_values(by=['diff'], ascending=False)
        df1.to_excel('d:/stockdata/vote_stock/analysis/'+select+i[48:59]+'.xlsx')

    print("time:", time.time()-start)
    
vote_stock('pprice')
#vote_stock('volume')

In [ ]:
###  vote-stock  기간동알 일별로 생성

from mod1 import *

start = time.time()
path_price = 'd:/stockdata/vote_stock/detect_stock_with_pprice_'
path_volume = 'd:/stockdata/vote_stock/detect_stock_with_volume_'

df  =all_stock_period('2019-01-01')
df = df[df['Volume'] >  500000]
df.reset_index(drop=True)
display(df)

day_list = []

for time in pd.date_range('2020-01-01', '2021-02-23', freq='B'):   ###   2020-01-01 ~ 2021-02-24 까지  영업일
    day_list.append(str(time)[:10])

for i in range(len(day_list)-1):
    from_day=  day_list[i]
    print(from_day)
    to_day = day_list[i+1]
    print(to_day)

    df1 = df[df['Date'].astype(str) == from_day]
    df1 = df1[['Name','Volume','Close']]
    df1.columns = ['Name','yester_Volume','yester_Close']    

    df2 = df[df['Date'].astype(str) == to_day]
    df2 = df2[['Name','Volume','Close']]
    df2.columns = ['Name','today_Volume','today_Close']

    df3 = pd.merge(df1,df2,on='Name')
    df3['Close']=df3['today_Close']/df3['yester_Close']
    df3['Volume']=df3['today_Volume']/df3['yester_Volume']
    df3 = df3.sort_values(by=['Volume','Close'],ascending=False)
    df4 = df3.sort_values(by=['Close','Volume'],ascending=False)
    df3 = df3.reset_index(drop=True)
    df4 = df4.reset_index(drop=True)
    df3 = df3[:15]
    df4 = df4[:15]
    df3.to_excel(path_volume+to_day+'.xlsx', encoding='utf-8')
    df4.to_excel(path_price+to_day+'.xlsx', encoding='utf-8')        
    display(df3)
    display(df4)
    
print('time:', time.time()-start)

In [ ]:
## stockdata / close_ma120 디렉토리 추이 분석 (멀티화일 처리) _1

from mod1 import *

def total_analysis(select, month, day):
    start = time.time()
    date_query = "select Date from market where Name='hrs' order by Date desc limit 1"

    df = pd.read_sql(date_query, engine)
    df = pd.to_datetime(df['Date'])
    df = str(df)
    last_date = df[4:14]                ## table에서 최종 날짜 추줄

    start = time.time()

    path = 'f:/stockdata/close_ma120/2020/'+month+'/'+day+'/'
    file = glob.glob(path+'total_filter'+'*.xlsx')
    print(file)
    for i in file:
        print(i)

    print("time:", time.time()-start)
    print(path)
total_analysis('total_filter','2020_01','0103')

# Graph 그리기

In [ ]:
##  DATABASE에서 각 Table별 그래프 그리기


def get_graph(choice=1):
    graph_name_list=['stock','money', 'program','future']
    date='2019-01-01'
    future_date='2019-12-11'  ##  선물마감 하루전

    if choice == 1:
        graph = input("그래프종류를 입력하세요 sample: 'money' or 'program' or 'stock' or 'future':  ")
        date = input("날짜를 입력하세요 sample: '2019-01-10':") or '2019-01-01'

        if graph == 'money' :
            money_name = ['kpi200', '거래량', '고객예탁금', '신용잔고']
            money_query = "select * from kpi_with_money where Date >"+"'"+date+"'"
            money_df = pd.read_sql(money_query ,engine)

            money_df.columns=['Date','kpi200', '거래량', '고객예탁금', '신용잔고', '주식형펀드', '혼합형펀드', '채권형펀드']
            money_df = money_df.set_index('Date')
            df1 = money_df[money_name]
            #return df1

            plt.figure(figsize=(16,4))         
            colors = ['red','green','blue','black']
            for i in range(len(money_name)):
                plt.subplot(2,2,i+1)
                plt.plot(df1[money_name[i]]/df1[money_name[i]].loc[money_df.index[0]]*100, color=colors[i])
                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                #plt.show()

        elif graph == 'program' :
            program_name = ['차익', '비차익', '전체']
            program_query = "select * from programtrend where Date >"+"'"+date+"'"
            program_df = pd.read_sql(program_query ,engine)

            program_df.columns=['Date','차익', '비차익', '전체']
            program_df = program_df.set_index('Date')
            df1=program_df[program_name]
            #return df1

            plt.figure(figsize=(16,4))        
            colors = ['red','green','blue','black']
            for i in range(len(program_name)):

                plt.subplot(2,2,i+1)
                plt.plot(df1[program_name[i]],color=colors[i])

                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                #plt.show()

        elif graph == 'stock' :
            name = input('주식이름을 입력하세요:').split()
            #date = input("날짜를 입력하세요 sample: '2019-01-10':")

            select_query = "select Date,Volume,Close from market where Name= "
            date_query = "Date > "


            tuple_name=tuple(name)
            df1 = pd.DataFrame()

            for x in tuple_name:
                var = select_query +"'"+x+"'"+" "+"&&"+" "+date_query+"'"+date+"'"
                df = pd.read_sql(var ,engine)
                df.columns=['Date',x+'거래량',x]
                if df1.empty:
                        df1 = df
                else:
                    df1 = pd.merge (df,df1,on='Date')
            df1=df1.set_index('Date')
            size = len(df1.index)

            plt.figure(figsize=(16,4))
            for i in range(len(name)):
                plt.plot(df1[name[i]]/df1[name[i]].loc[df['Date'][0]]*100)

                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)

            plt.figure(figsize=(16,4))
            for i in range(len(name)):
                volume_average = df1[name[i]+'거래량'].sum(axis=0)/size
                plt.plot(df1[name[i]+'거래량']/volume_average)
                #plt.plot(df1[name[i]+'거래량']/df1[name[i]+'거래량'].loc[df['Date'][0]]*100, label =[name[i]+'거래량'] )
                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                    
        elif graph == 'future' :

            #name = input("항목을 입력하세요: 선택항목: 'kpi200', '거래량', '고객예탁금', '신용잔고', '주식형펀드', '혼합형펀드', '채권형펀드'").split()
            #date = input("날짜를 입력하세요 sample: '2019-01-10':")

            #query = "select * from future where Date > '2019-06-13'"+"'"+date+"'"
            query = "select * from future where Date >"+"'"+future_date+"'"
            query1 = "select * from basis where Date >"+"'"+future_date+"'"

            name=['Close', '미결제약정', '외국인', '기관', '개인']
            name1=['Close','미결제약정']
            name2=['외국인', '기관', '개인']
            basis_name=['kpi200','Future']

            #tuple_name=tuple(name)
            df1 = pd.DataFrame()
            basis_df1 = pd.DataFrame()

            df = pd.read_sql(query ,engine)
            basis_df = pd.read_sql(query1 ,engine)

            df.columns=['Date', 'Close', '미결제약정', '외국인', '기관', '개인']
            df = df.set_index('Date')
            df1=df[name]

            basis_df = basis_df.set_index('Date')
            basis_df1=basis_df[basis_name]

            colors = ['red','green','blue','black']
            plt.figure(figsize=(16,4))    
            for i in range(len(basis_name)):
                plt.plot(basis_df1[basis_name[i]]/basis_df1[basis_name[i]].loc[basis_df.index[0]]*100)

            plt.legend(loc=0)
            plt.grid(True,color='0.7',linestyle=':',linewidth=1)
            plt.show()
                
            plt.figure(figsize=(16,4))    
            for i in range(len(name1)):
                #plt.subplot(2,2,i+1)
                plt.plot(df1[name1[i]]/df1[name1[i]].loc[df.index[0]]*100)

            plt.legend(loc=0)
            plt.grid(True,color='0.7',linestyle=':',linewidth=1)
            plt.show()

            plt.figure(figsize=(16,4)) 
            for i in range(len(name2)):
                plt.subplot(2,2,i+1)
                plt.plot(df1[name2[i]]/df1[name2[i]].loc[df.index[0]]*100,color = colors[i])

                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
 
        else : 
            print('\n input error\n')

                
    else :
            
        bokeh_chart('kospi','2019-01-01','month')
        bokeh_chart('kospi','2020-01-01','week')
        df1 = select_market('kospi','2020-01-01')
        market_ma(df1,'ma5','ma20')
        bokeh_chart('kosdaq','2019-01-01','month')
        bokeh_chart('kosdaq','2020-01-01','week')
        df1 = select_market('kosdaq','2020-01-01')
        market_ma(df1,'ma5','ma20')
            
        for i in three_period:
            depress(i)
            
        kpi200_df = pd.read_sql("select Date from market where Name='hrs' order by Date desc limit 2", engine)
        yesterday = str(kpi200_df['Date'][1])
        today = str(kpi200_df['Date'][0])
            
        for i in graph_name_list:
            if i == 'stock' :
                name = pd.read_excel(path_volume+today+'.xlsx', encoding='utf-8')
                name_all = name['Name']
                name_all = name_all.to_list()
                name = name[:5]
                name = name['Name']
                name = name.to_list()

                select_query = "select Date,Volume,Close from market where Name= "
                date_query = "Date > "


                tuple_name=tuple(name)
                df1 = pd.DataFrame()
                
                for x in tuple_name:
                    var = select_query +"'"+x+"'"+" "+"&&"+" "+date_query+"'"+date+"'"
                    df = pd.read_sql(var ,engine)
                    df.columns=['Date',x+'거래량',x]
                    if df1.empty:
                        df1 = df
                    else:
                        df1 = pd.merge (df,df1,on='Date')
                df1=df1.set_index('Date')
                size = len(df1.index)


                plt.figure(figsize=(16,4))
                for i in range(len(name)):
                    try:                    ###  종목별 그래프 그리는도종  input NoNs 에러 방지
                        plt.plot(df1[name[i]]/df1[name[i]].loc[df['Date'][0]]*100,label=name[i])
                        plt.legend(loc=0)
                        plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                    except:
                        pass

                plt.figure(figsize=(16,4))
                for i in range(len(name)):
                    volume_average = df1[name[i]+'거래량'].sum(axis=0)/size
                    plt.plot(df1[name[i]+'거래량']/volume_average, label=name[i])
                    #plt.plot(df1[name[i]+'거래량']/df1[name[i]+'거래량'].loc[df['Date'][0]]*100, label =[name[i]+'거래량'] )
                    plt.legend(loc=0)
                    plt.grid(True,color='0.7',linestyle=':',linewidth=1)  

                for i in name:
                    var = select_query +"'"+i+"'"+" "+"&&"+" "+date_query+"'"+date+"'" 
                    df = pd.read_sql(var, engine)
                    df[['Volume','Close']] = df[['Volume','Close']].astype(float) #  TA-Lib로 평균을 구하려면 실수로 만들어야 함
                    df.columns=df.columns.str.lower()
                        
                    talib_ma120 = ta.MA(df, timeperiod=120)
                    df['ma120'] = talib_ma120
                        
                    source = MinMaxScaler()
                    data = source.fit_transform(df[['close','volume','ma120']].values)
                    df1 = pd.DataFrame(data)
                    df1.columns=['close','volume','ma120']
                    df1 = df1.set_index(df['date'])
                    df1.plot(figsize=(16,2))
                    plt.title(i)
                    plt.show()
                
            elif i == 'money' :
                money_name = ['kpi200', '거래량', '고객예탁금', '신용잔고']
                money_query = "select * from kpi_with_money where Date >"+"'"+date+"'"
                money_df = pd.read_sql(money_query ,engine)

                money_df.columns=['Date','kpi200', '거래량', '고객예탁금', '신용잔고', '주식형펀드', '혼합형펀드', '채권형펀드']
                money_df = money_df.set_index('Date')
                df1 = money_df[money_name]
                #return df1

                plt.figure(figsize=(16,4))         
                colors = ['red','green','blue','black']
                for i in range(len(money_name)):
                    plt.subplot(2,2,i+1)
                    plt.plot(df1[money_name[i]]/df1[money_name[i]].loc[money_df.index[0]]*100, color=colors[i],label=money_name[i])
                    plt.legend(loc=0)
                    plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                    #plt.show()

            elif i == 'program' :
                program_name = ['차익', '비차익', '전체']
                program_query = "select * from programtrend where Date >"+"'"+date+"'"
                program_df = pd.read_sql(program_query ,engine)

                program_df.columns=['Date','차익', '비차익', '전체']
                program_df = program_df.set_index('Date')
                df1=program_df[program_name]
                #return df1

                plt.figure(figsize=(16,4))        
                colors = ['red','green','blue','black']
                for i in range(len(program_name)):

                    plt.subplot(2,2,i+1)
                    plt.plot(df1[program_name[i]],color=colors[i],label=program_name[i])

                    plt.legend(loc=0)
                    plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                    #plt.show()
                        
            elif i == 'future' :
                query = "select * from future where Date >"+"'"+future_date+"'"
                query1 = "select * from basis where Date >"+"'"+future_date+"'"
                name=['Close', '미결제약정', '외국인', '기관', '개인']
                name1=['Close','미결제약정']
                name2=['외국인', '기관', '개인']
                basis_name=['kpi200','Future']

                df1 = pd.DataFrame()
                basis_df1 = pd.DataFrame()

                df = pd.read_sql(query ,engine)
                basis_df = pd.read_sql(query1 ,engine)

                df.columns=['Date', 'Close', '미결제약정', '외국인', '기관', '개인']
                df = df.set_index('Date')
                df1=df[name]

                basis_df = basis_df.set_index('Date')
                basis_df1=basis_df[basis_name]

                colors = ['red','green','blue','black']
                plt.figure(figsize=(16,4))    
                for i in range(len(basis_name)):
                    plt.plot(basis_df1[basis_name[i]]/basis_df1[basis_name[i]].loc[basis_df.index[0]]*100,label=basis_name[i])

                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                plt.show()

                plt.figure(figsize=(16,4))    
                for i in range(len(name1)):
                    plt.plot(df1[name1[i]]/df1[name1[i]].loc[df.index[0]]*100,label=name1[i])

                plt.legend(loc=0)
                plt.grid(True,color='0.7',linestyle=':',linewidth=1)
                plt.show()

                plt.figure(figsize=(16,4)) 
                for i in range(len(name2)):
                    plt.subplot(2,2,i+1)
                    plt.plot(df1[name2[i]]/df1[name2[i]].loc[df.index[0]]*100,color = colors[i],label=name2[i])

                    plt.legend(loc=0)
                    plt.grid(True,color='0.7',linestyle=':',linewidth=1)

get_graph(choice=2)                    

# Graph 그리기 - inputs all NaNs error 건너뛰기

In [ ]:
###  MINMax로 표준화하여 그래프 생성

##  def close_ma_vol 에서 tyr , except 사용  *****   inputs all NaNs error 건너뛰기 ******


from mod1 import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

path = 'f:/stockdata/close_ma120/total_filter'
path_total_f = 'f:\\stockdata\\close_ma120\\2020\\2020-08\\0811\\total_filter_'
path_price = 'f:\\stockdata\\vote_stock\\detect_stock_with_price_'
path_volume = 'f:\\stockdata\\vote_stock\\detect_stock_with_volume_'
path_depress_d = 'f:\\stockdata\\depress\\depress_day_'
path_depress_w = 'f:\\stockdata\\depress\\depress_week_'
path_depress_m = 'f:\\stockdata\\depress\\depress_month_'



part = path_total_f
day = '2020-02-20'
from_date = '2018-01-01'
to_date = '2020-01-04'


def select(part, day):
    choice_date = day
    file = pd.read_excel(part+choice_date+'.xlsx')
    file = file[['name_x','price_x']]
    file = file.rename(columns={'name_x':'Name','price_x':'Closed'})
    name = file['Name']
    for i in name:
        #df = select_stock_period(i,'2008-01-01','2020-01-02')
        df = select_stock_period(i,from_date, to_date)
        #close_ma(df,'ma60','ma120')
        #rsi(df)
        #obv(df)
        close_ma_vol(df,'ma60','ma120','volume')

def select_stock_period(name, from_date, to_date):
    #from_date  = '2020-01-01'
    #to_date = '2020-02-01'
    select_query = "select * from market where Name= "
    from_query = "Date > "
    to_query = "Date < "
    #query = select_query +"'"+name+"'"+" "+"and"+" "+from_query+"'"+from_date+"'"+" "+"and"+" "+to_query+"'"+to_date+"'"
    query = select_query +"'"+name+"'"+" "+"and"+" "+from_query+"'"+from_date+"'"+" "+"and"+" "+to_query+"'"+to_date+"'"
    df = pd.read_sql(query, engine)
    return df
        

def close_ma(df,select1,select2):
        maa(df)

        source = MinMaxScaler()
        data = source.fit_transform(df[['close',select1,select2]].values)
        df1 = pd.DataFrame(data)
        df1.columns=['close',select1,select2]
        df1 = df1.set_index(df['date'])
        df1.iloc[14:].plot(grid=True,figsize=(16,4));
        plt.title(df['name'][0])
        plt.show()


def close_ma_vol(df,select1,select2,select3):
    try:                                    ##  inputs이 NaN으로 에러날때 건너뛰기
        ma(df)
        source = MinMaxScaler()
        data = source.fit_transform(df[['close',select1,select2,select3]].values)
        df1 = pd.DataFrame(data)
        df1.columns=['close',select1,select2,select3]
        df1 = df1.set_index(df['date'])
        df1.iloc[14:].plot(grid=True,figsize=(16,4));
        plt.title(df['name'][0])
        plt.show()
        
    except:                               ##  inputs이 NaN으로 에러날때 건너뛰기
        pass


    
def rsi(df):
    df = df.set_index('date')
    talib_rsi = ta.RSI(df, timeperiod=14)
    talib_rsi.iloc[14:].plot( grid=True,figsize=(16,4))
    plt.fill_between(df.index,y1=30, y2=70, color='#adccff', alpha='0.3')
    plt.show()
    
def obv(df):
    df = df.set_index('date')
    real = ta.OBV(df)
    real.iloc[14:].plot( y=['volume'], grid=True,figsize=(16,4));
    plt.show()


select(part, day)



In [ ]:
##  def close_ma_vol 에서 tyr , except 사용  *****   inputs all NaNs error 건너뛰기 ******
##  종류별(vote_stock, depress, total_filter) data를 동일한포맷으로 graph 그리기


from mod1 import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

path = 'f:/stockdata/close_ma120/total_filter'
path_total_f = 'f:\\stockdata\\close_ma120\\2020\\2020-08\\0811\\total_filter_'
path_price = 'f:\\stockdata\\vote_stock\\detect_stock_with_price_'
path_volume = 'f:\\stockdata\\vote_stock\\detect_stock_with_volume_'
path_depress_d = 'f:\\stockdata\\depress\\depress_day_'
path_depress_w = 'f:\\stockdata\\depress\\depress_week_'
path_depress_m = 'f:\\stockdata\\depress\\depress_month_'
path_v = 'f:\\stockdata\\vote_stock\\analysis\\volume_'
path_p = 'f:\\stockdata\\vote_stock\\analysis\\pprice_'


part = path_depress_w
day = '2020-09-03'
from_date = '2018-01-01'
to_date = '2020-08-27'
select1 = 'ma60'
select2 = 'ma120'

def select(part, day):
    df = pd.read_excel(part+day+'.xlsx')
    name = df['Name']          
    for i in name:
        df = select_stock_period(i,from_date, to_date)
        #close_ma(df,select1,select2)
        close_ma_vol(df,'ma60','ma120','volume')

def select_stock_period(name, from_date, to_date):
    #from_date  = '2020-01-01'
    #to_date = '2020-02-01'
    select_query = "select * from market where Name= "
    from_query = "Date > "
    to_query = "Date < "
    #query = select_query +"'"+name+"'"+" "+"and"+" "+from_query+"'"+from_date+"'"+" "+"and"+" "+to_query+"'"+to_date+"'"
    query = select_query +"'"+name+"'"+" "+"and"+" "+from_query+"'"+from_date+"'"+" "+"and"+" "+to_query+"'"+to_date+"'"
    df = pd.read_sql(query, engine)
    return df
        

def close_ma_vol(df,select1,select2,select3):
    try:                                    ##  inputs이 NaN으로 에러날때 건너뛰기
        ma(df)
        source = MinMaxScaler()
        data = source.fit_transform(df[['close',select1,select2,select3]].values)
        df1 = pd.DataFrame(data)
        df1.columns=['close',select1,select2,select3]
        df1 = df1.set_index(df['date'])
        df1.iloc[14:].plot(grid=True,figsize=(16,4));
        plt.title(df['name'][0])
        plt.show()
        
    except:                               ##  inputs이 NaN으로 에러날때 건너뛰기
        pass
    
    
def close_ma(df,select1,select2):
    try:                                    ##  inputs이 NaN으로 에러날때 건너뛰기
        ma(df)
        #source = MinMaxScaler()
        #data = source.fit_transform(df[['close',select1,select2,select3]].values)
        #df1 = pd.DataFrame(data)
        df1 = df[['close',select1,select2]]
        
        df1 = df1.set_index(df['date'])
        df1.iloc[14:].plot(grid=True,figsize=(16,4));
        plt.title(df1['Name'][0])
        plt.show()
        
    except:                               ##  inputs이 NaN으로 에러날때 건너뛰기
        pass

select(part, day)

In [ ]:
##  시작날짜를 100%로 두고 변화를 확인

from mod1 import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

path = 'd:/stockdata/close_ma120/total_filter'
path_total_f = 'd:/stockdata/close_ma120/2020/2020-08/0811/total_filter_'
path_price = 'd:/stockdata/vote_stock/detect_stock_with_pprice_'
path_volume = 'd:/stockdata/vote_stock/detect_stock_with_volume_'
path_depress_d = 'd:/stockdata/depress/depress_day_'
path_depress_w = 'd:/stockdata/depress/depress_week_'
path_depress_m = 'd:/stockdata/depress/depress_month_'
path_v = 'd:/stockdata/vote_stock/analysis/volume_'
path_p = 'd:/stockdata/vote_stock/analysis/pprice_'

part = path_volume
day = '2020-08-07'
from_date = '2018-01-01'
to_date = '2020-08-27'
select1 = 'ma60'
select2 = 'ma120'
date=day
future_date='2019-12-11'  ##  선물마감 하루전

df = pd.read_excel(part+day+'.xlsx', encoding='utf-8')
name = df['Name']
name = name[:5]
select_query = "select Date,Volume,Close from market where Name= "
date_query = "Date > "

tuple_name=tuple(name)
df1 = pd.DataFrame()

for x in tuple_name:
    var = select_query +"'"+x+"'"+" "+"&&"+" "+date_query+"'"+date+"'"
    df = pd.read_sql(var ,engine)
    df.columns=['Date',x+'거래량',x]
    if df1.empty:
        df1 = df
    else:
        df1 = pd.merge (df,df1,on='Date')
    df1=df1.set_index('Date')
    size = len(df1.index)

plt.figure(figsize=(16,4))
for i in range(len(name)):
    try:
        plt.plot(df1[name[i]]/df1[name[i]].loc[df['Date'][0]]*100,label=name[i])
        plt.legend(loc=0)
        plt.grid(True,color='0.7',linestyle=':',linewidth=1)
    except:
        pass

plt.figure(figsize=(16,4))
for i in range(len(name)):
    try:
        volume_average = df1[name[i]+'거래량'].sum(axis=0)/size
        plt.plot(df1[name[i]+'거래량']/volume_average, label=name[i])
        #plt.plot(df1[name[i]+'거래량']/df1[name[i]+'거래량'].loc[df['Date'][0]]*100, label =[name[i]+'거래량'] )
        plt.legend(loc=0)
        plt.grid(True,color='0.7',linestyle=':',linewidth=1)
    except:
        pass 

In [ ]:
###  dataframe내에 있는 종목별로 분류하여 graph 생성

from mod1 import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

path = 'd:/stockdata/close_ma120/total_filter'
path_total_f = 'd:/stockdata/close_ma120/2020/2020-08/0811/total_filter_'
path_price = 'd:/stockdata/vote_stock/detect_stock_with_pprice_'
path_volume = 'd:/stockdata/vote_stock/detect_stock_with_volume_'
path_depress_d = 'd:/stockdata/depress/depress_day_'
path_depress_w = 'd:/stockdata/depress/depress_week_'
path_depress_m = 'd:/stockdata/depress/depress_month_'
path_v = 'd:/stockdata/vote_stock/analysis/volume_'
path_p = 'd:/stockdata/vote_stock/analysis/pprice_'

count = 5
part = path_price
day = '2020-08-06'
from_date = '2018-01-01'
to_date = '2020-08-27'
select1 = 'ma60'
select2 = 'ma120'
date=day
future_date='2019-12-11'  ##  선물마감 하루전

df = pd.read_excel(part+day+'.xlsx', encoding='utf-8')
name = df['Name']
name = name[:10]
select_query = "select Date,Volume,Close from market where Name= "
date_query = "Date > "

tuple_name=tuple(name)
df1 = pd.DataFrame()

for x in name:
    df = select_stock(x,day)
    num = df.shape[0]/count
    for i in range(int(num)):
        df1 = df1.append(df.iloc[i*count])
name = df1['Name'].unique()
for i in name:
    df2 = df1[df1['Name']== i]
    plt.figure(figsize=(16,4))
    plt.plot(df2.index,df2['Close'])
    plt.title(df2['Name'][0])
    plt.grid(True)
    plt.show()    

In [ ]:
##### Close Graph 비교를 표준화

def option(path=path_volume, day=real_today, graph_start_date='2020-01-01',count=5):
    name = pd.read_excel(path_volume+day+'.xlsx')
    name = name[:count]
    name = name['Name']
    #name = name.to_list()
    return name,graph_start_date


def compare_graph(method):
    name, graph_start_date = method
    #name = ['hrs','오공','모트렉스']
    #name = input('주식이름을 입력하세요:').split()
    #date = input("날짜를 입력하세요 sample: '2019-01-10': ")

    select_query = "select Date,Close from market where Name= "
    date_query =  "Date >"

    tuple_name=tuple(name)
    df1 = pd.DataFrame()

    for x in tuple_name:
        var = select_query +"'"+x+"'"+" "+"&&"+" "+date_query+"'"+graph_start_date+"'"
        df = pd.read_sql(var ,engine) 
        df.columns=['Date',x]
        if df1.empty:
            df1 = df
        else:
            df1 = pd.merge (df,df1,on='Date')
    df1=df1.set_index('Date')

    plt.figure(figsize=(12,5))

    for i in range(len(name)):
        plt.plot(df1[name[i]]/df1[name[i]].loc[df['Date'][0]]*100)
        plt.legend(name,loc=0)
        #plt.legend(loc=0,['hrs','오공','모트렉스'])
        plt.grid(True,color='0.7',linestyle=':',linewidth=2)
        
compare_graph(option(day='2020-10-23'))

In [ ]:
##### Close , Volume Graph 비교를 표준화
from mod1 import *
path_depress_d = 'f:\\stockdata\\depress\\depress_day_'
def option(path=path_volume, day=real_today, graph_start_date='2020-01-01',start_count=0):
    end_count=start_count+5
    name = pd.read_excel(path+day+'.xlsx')
    name = name[start_count:end_count]
    name = name['Name']
    name = name.to_list()
    return name,graph_start_date


def compare_graph(method):
    name, graph_start_date = method
    #name = ['hrs','오공','모트렉스']
    #name = input('주식이름을 입력하세요:').split()
    #date = input("날짜를 입력하세요 sample: '2019-01-10': ")

    select_query = "select Date,Close,Volume from market where Name= "
    date_query =  "Date >"

    tuple_name=tuple(name)
    df_Close = pd.DataFrame()
    df_Volume = pd.DataFrame()
    dfc = pd.DataFrame()
    dfv = pd.DataFrame()

    for x in name:
        var = select_query +"'"+x+"'"+" "+"&&"+" "+date_query+"'"+graph_start_date+"'"
        df = pd.read_sql(var ,engine)
        df_Close = df[['Date', 'Close']]
        df_Close.columns=['Date',x]
        df_Close = df_Close.set_index('Date')
        dfc = pd.concat([dfc,df_Close], axis=1)
        df_Volume = df[['Date', 'Volume']]
        df_Volume.columns=['Date',x]
        df_Volume = df_Volume.set_index('Date')
        dfv = pd.concat([dfv,df_Volume], axis=1)

    plt.figure(figsize=(12,5))

    for i in range(len(name)):
        plt.plot(dfc[name[i]]/dfc[name[i]].loc[df['Date'][0]]*100)
        #plt.plot(dfv[name[i]]/dfv[name[i]].loc[df['Date'][0]]*100)        
    plt.legend(name,loc=0)
    plt.grid(True,color='0.7',linestyle=':',linewidth=1)
    plt.figure(figsize=(12,5))
    for i in range(len(name)):
        #plt.plot(dfc[name[i]]/dfc[name[i]].loc[df['Date'][0]]*100)
        plt.plot(dfv[name[i]]/dfv[name[i]].loc[df['Date'][0]]*100)        
    plt.legend(name,loc=0)
    plt.grid(True,color='0.7',linestyle=':',linewidth=1)


#option(day='2020-10-23')

compare_graph(option(path=path_depress_d,day='2020-10-22',start_count=6))


In [ ]:
from mod1 import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

path = 'd:/stockdata/close_ma120/close_ma120_/'
path_total_f = 'd:/stockdata/close_ma120/2020/2020-07/0709/total_filter_'
path_price = 'd:/stockdata/vote_stock/detect_stock_with_price_'
path_volume = 'd:/stockdata/vote_stock/detect_stock_with_volume_'

choice_date='2020-05-13'
df_volume = pd.read_excel(path_price+choice_date+'.xlsx')
name_volume = df_volume['Name']
#name_volume = df_volume['name_x']

def close_ma(df,select1,select2):
    ma(df)

    source = MinMaxScaler()
    data = source.fit_transform(df[['close',select1,select2]].values)
    df1 = pd.DataFrame(data)
    df1.columns=['close',select1,select2]
    df1 = df1.set_index(df['date'])
    df1.iloc[14:].plot(grid=True,figsize=(16,4));
    plt.title(df['name'][0])
    plt.show()

def close_ma_vol(df,select1,select2,select3):
    ma(df)

    source = MinMaxScaler()
    data = source.fit_transform(df[['close',select1,select2,select3]].values)
    df1 = pd.DataFrame(data)
    df1.columns=['close',select1,select2,select3]
    df1 = df1.set_index(df['date'])
    df1.iloc[14:].plot(grid=True,figsize=(16,4));
    plt.title(df['name'][0])
    plt.show()    
    
def rsi(df):
    df = df.set_index('date')
    talib_rsi = ta.RSI(df, timeperiod=14)
    talib_rsi.iloc[14:].plot( grid=True,figsize=(16,4))
    plt.fill_between(df.index,y1=30, y2=70, color='#adccff', alpha='0.3')
    plt.show()
    
def obv(df):
    df = df.set_index('date')
    real = ta.OBV(df)
    real.iloc[14:].plot( y=['volume'], grid=True,figsize=(16,4));
    plt.show()

for i in name_volume:
    df = select_stock(i,'2010-01-01')
    #close_ma(df,'ma60','ma120')
    #rsi(df)
    #obv(df)
    close_ma_vol(df,'ma60','ma120','volume')


In [ ]:
from mod1 import *

path = 'f:\\stockdata\\close_ma120\\close_ma120_\\'
path_total_f = 'f:\\stockdata\\close_ma120\\2020\\2020-07\\0709\\total_filter_'
path_price = 'f:\\stockdata\\vote_stock\\detect_stock_with_price_'
path_volume = 'f:\\stockdata\\vote_stock\\detect_stock_with_volume_'
os.listdir(path_volume)



In [ ]:
from mod1 import *

path='f:/stockdata/close_ma120/2020/2020-08/0805/'
os.listdir(path)



In [ ]:
df = pd.read_excel(path+'total_filter_2020-02-20.xlsx')
name=df['name_x'].to_list()

for i in name:
    df = select_stock(i, '2010-01-01')
    
    close_ma(df,'ma60','ma120')

In [ ]:
dir(mod1.py)

In [ ]:
dd

In [ ]:
from mod1 import *
import seaborn as sns
sns.set()

df = select_stock('아난티','2019-01-01')

count = int(np.ceil(len(df) * 0.1))
signals = pd.DataFrame(index=df.index)
signals['signal'] = 0.0
signals['trend'] = df['Close']
signals['RollingMax'] = (signals.trend.shift(1).rolling(count).max())
signals['RollingMin'] = (signals.trend.shift(1).rolling(count).min())
signals.loc[signals['RollingMax'] < signals.trend, 'signal'] = -1
signals.loc[signals['RollingMin'] > signals.trend, 'signal'] = 1
signals

In [ ]:
def buy_stock(
    real_movement,
    signal,
    initial_money = 1000000,
    max_buy = 1,
    max_sell = 1,
):
    """
    real_movement = actual movement in the real world
    delay = how much interval you want to delay to change our decision from buy to sell, vice versa
    initial_state = 1 is buy, 0 is sell
    initial_money = 1000, ignore what kind of currency
    max_buy = max quantity for share to buy
    max_sell = max quantity for share to sell
    """
    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0

    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        if shares < 1:
            print(
                'day %d: total balances %f, not enough money to buy a unit price %f'
                % (i, initial_money, real_movement[i])
            )
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(
                'day %d: buy %d units at price %f, total balance %f'
                % (i, buy_units, buy_units * real_movement[i], initial_money)
            )
            states_buy.append(0)
        return initial_money, current_inventory

    for i in range(real_movement.shape[0] - int(0.025 * len(df))):
        state = signal[i]
        if state == 1:
            initial_money, current_inventory = buy(
                i, initial_money, current_inventory
            )
            states_buy.append(i)
        elif state == -1:
            if current_inventory == 0:
                    print('day %d: cannot sell anything, inventory 0' % (i))
            else:
                if current_inventory > max_sell:
                    sell_units = max_sell
                else:
                    sell_units = current_inventory
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                try:
                    invest = (
                        (real_movement[i] - real_movement[states_buy[-1]])
                        / real_movement[states_buy[-1]]
                    ) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (i, sell_units, total_sell, invest, initial_money)
                )
            states_sell.append(i)
            
    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest

In [ ]:
states_buy, states_sell, total_gains, invest = buy_stock(df.Close, signals['signal'])

In [ ]:
invest

# 강화학습(Reinforcement learning) 

In [ ]:
from mod1 import *

df = select_stock('오공', '2015-01-01')
df = df.iloc[:1258]

training_set=df.iloc[:,3:4].values

from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler()
training_set=sc.fit_transform(training_set)

X_train= training_set[0:1257]
y_train= training_set[1:1258]

X_train=np.reshape(X_train, (1257 , 1 , 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

regressor = Sequential()

regressor.add(LSTM(units=4, activation= 'sigmoid', input_shape= (None,1)))

regressor.add(Dense( units=1 ))

regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(X_train, y_train, batch_size=32, epochs=200)

# Data 가공

In [ ]:
##  기준 Data를 시초가-고가,  시초가-저가, 시초가-종가, 종가-고가, 종가-저가, 전날거래량과 차이등 가공

from mod1 import *

df = pd.read_sql("select * from market where Name='hrs' and Date > '2020-12-01'",engine)
df['shift_P'] = df['Close'].shift()
df['shift_V'] = df['Volume'].shift()
df['shift_P'][0]=df['Close'][0]
df['shift_V'][0]=df['Volume'][0]
df['slope_P']=df['shift_P']/df['Close']
df['slope_V']=df['shift_V']/df['Volume']
df['OH']=df['High']/df['Open']
df['OL']=df['Low']/df['Open']
df['OC']=df['Close']/df['Open']
df['CH']=df['High']/df['Close']
df['CL']=df['Low']/df['Close']
df

# Prophet ( forecast : 예측) module

In [ ]:
#  stock _ prediction _1


from mod1 import *
import plaidml.keras
plaidml.keras.install_backend()

name = 'lg전자'
front =1200
back =1201

start = time.time()  # 시작 시간 저장
df = select_stock(name,'2015-01-01')
df = df.set_index(df['Date'])
df1 = df.iloc[:back]
df1 = df1.set_index(df1['Date'])
df2 = df.iloc[back:]
df2= df2.set_index(df2['Date'])

training_set=df1.iloc[:,3:4].values
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler()
training_set=sc.fit_transform(training_set)

X_train= training_set[0:front]
y_train= training_set[1:back]

X_train=np.reshape(X_train, (front , 1 , 1))
maxlen=1257

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

regressor = Sequential()
regressor.add(LSTM(units=4, activation= 'relu',input_shape=( 1,1)))
regressor.add(Dense( units=1 ))
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(X_train, y_train, batch_size=32, epochs=300)

real_stock_price = df2.iloc[:,3:4].values

inputs = real_stock_price
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (df2.shape[0] , 1, 1))
predicted_stock_price = regressor.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

plt.figure(figsize=(16,4))
plt.plot(df2.index,real_stock_price , color = 'red' , label = 'Real Stock Price')
plt.plot(df2.index,predicted_stock_price , color = 'blue' , label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel( 'time' )
plt.ylabel( name+'Stock Price' )
plt.legend()
plt.show()
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
##   stock _ prediction _1 함수화

from mod1 import *
import plaidml.keras
plaidml.keras.install_backend()

name = 'lg전자'
front =200
back =201
sel_open = 3
sel_high = 4
sel_low=5
sel_volume=6
sel_close=7
activation_by='relu'
optimizer_by = 'adam'
loss_by = 'mean_squared_error'
batch=32
epoch=300

start = time.time()  # 시작 시간 저장
df = select_stock(name,'2015-01-01')
df = df.set_index(df['Date'])
df1 = df.iloc[:back]
df1 = df1.set_index(df1['Date'])
df2 = df.iloc[back:]
df2= df2.set_index(df2['Date'])
sc= MinMaxScaler()

def make_data( ):
    training_set=df1.iloc[:,sel_open:sel_high].values
    training_set=sc.fit_transform(training_set)

    X_train= training_set[0:front]
    y_train= training_set[1:back]

    X_train=np.reshape(X_train, (front , 1 , 1))
    return X_train, y_train

def make_lstm( ):
    regressor = Sequential()
    regressor.add(LSTM(units=18, return_sequences=True, activation= activation_by,input_shape=( 1,1)))
    regressor.add(LSTM(units=4))
    regressor.add(Dense( units=1 ))
    return regressor


def make_study( ):
    regressor.compile(optimizer=optimizer_by, loss=loss_by)
    model = regressor.fit(X_train, y_train, batch_size=batch, epochs=epoch)
    


X_train, y_train = make_data( )
regressor = make_lstm( )
make_study( )

##  predict

real_stock_price = df2.iloc[:,3:4].values

inputs = real_stock_price
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (df2.shape[0] , 1, 1))
predicted_stock_price = regressor.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

##  make_graph
plt.figure(figsize=(16,4))
plt.plot(df2.index,real_stock_price , color = 'red' , label = 'Real Stock Price')
plt.plot(df2.index,predicted_stock_price , color = 'blue' , label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel( 'time' )
plt.ylabel( name+'Stock Price' )
plt.legend()
plt.show()
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
##  stock_prediction_2

from mod1 import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
import time 
import math
import seaborn as sb
from sklearn.model_selection import train_test_split


#df=pd.read_csv(input_file)
df = select_stock('삼성전자', '2015-01-01')
np.random.seed(5)
df.head()
df= df.set_index(df['Date'])


### Closing prices of stock

close_price=df.iloc[:,7].values
close_price=close_price.reshape(-1,1)
close_price

### Scaling values between 0 & 1. Machine Learning algorithms work better when features are on relatively similar scale & closer to normal distribution

scaler = MinMaxScaler(feature_range=(0, 1))
close_price = scaler.fit_transform(close_price)

def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        dataX.append(dataset[i:(i+look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

### Reshape into X=t and Y=t+1, timestep 240

train_size = int(len(close_price) * 0.6)
test_size = len(close_price) - train_size
train, test = close_price[0:train_size,:], close_price[train_size:len(close_price),:]
train, test = close_price[0:train_size,:], close_price[train_size:len(close_price),:]
look_back = 240
df2 = df.iloc[train_size+look_back+1:len(close_price)]
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

list(zip(trainX,trainY))

### Reshape input to be [samples, time steps, features]

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### Building LSTM model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback

model =Sequential([
LSTM(25, input_shape=(1, look_back)),
Dropout(0.1),
Dense(1)
])

model.summary()

### Setting optimizer, loss_fn & accuracy

model.compile(loss='mse', optimizer='adam',metrics=['mse'])

ACCURACY_THRESH=0.90

class myCallback(Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('acc')>=ACCURACY_THRESH:
            print('Desired accuracy reached.....stopping training')
            self.model.stop_training=True

callbacks=myCallback()

model.fit(trainX, trainY, epochs=300,batch_size=240, verbose=1)

### Saving the trained model

from keras.models import load_model
model.save('trained_model.h5')  # creates a HDF5 file 'my_model.h5'

#Load back the trained model
#model = load_model('my_model.h5')

### Predictions & Visualization

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

plt.figure(figsize=(16,4))
plt.plot(df2.index,testPredict,color='g')
plt.plot(df2.index,testY.reshape(-1,1),color='orange')
plt.ylabel('Closing Price')
plt.xlabel('Time Interval')
plt.legend(["Predicted", "Original"], loc ="upper left") 
plt.show()
sb.set()

In [ ]:
### https://programtalk.com/python-examples/backtrader.indicators.SimpleMovingAverage/
    
    
from mod1 import *
import datetime
import backtrader as bt
from fbprophet import Prophet

# Create a Stratey
class FirstStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )
 
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        
class SecondStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )
 
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
 
    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)
        
        
class ThirdStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )
 
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
            
class FourthStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )
 
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
 
    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

class FifthStrategy(bt.Strategy):
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # keep array of dates and closes
        self.date_array = []
        self.close_array = []

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.datas[0].close[0])
        
        # append date and close to arrays
        self.date_array.append(self.datas[0].datetime.date(0))
        self.close_array.append(self.datas[0].close[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # make sure we have a decent amount of data
        if len(self.date_array) < 90:
            return
        
        # only invest once a week
        if len(self.date_array) % 5 != 0:
            return
        
        # get predictions
        max_move, expected_move, min_move = self.get_prophet_moves(7, False)
        
        # if the predicted movement is mostly positive, buy
        if max_move > 0 and abs(max_move) > abs(min_move):
            self.log('BUY CREATE, %.2f' % self.datas[0].close[0])

            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()
        
        # if the predicted movement is mostly negative, sell
        elif min_move < 0 and abs(min_move) > abs(max_move):
            
            # make sure we have some stock to sell
            if self.position:
                self.log('SELL CREATE, %.2f' % self.datas[0].close[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def get_prophet_moves(self, daysOut=7, showCharts=False):
        # create stock dataframe for prophet
        stock_df = pd.DataFrame({
            'ds': self.date_array,
            'y': self.close_array
        })

        # fit data using prophet model
        m = Prophet()
        m.fit(stock_df)

        # create future dates
        future_prices = m.make_future_dataframe(periods=365)

        # predict prices
        forecast = m.predict(future_prices)

        # view results
        if showCharts:
            fig = m.plot(forecast)
            ax1 = fig.add_subplot(111)
            ax1.set_title("Stock Price Forecast", fontsize=16)
            ax1.set_xlabel("Date", fontsize=12)
            ax1.set_ylabel("Close Price", fontsize=12)

            fig2 = m.plot_components(forecast)
            plt.show()

        # calculate predicted returns
        end_of_period = self.datas[0].datetime.date(0) + dt.timedelta(days=daysOut)

        future_close_max = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_upper
        future_close_expected = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat
        future_close_min = forecast.loc[forecast.ds > end_of_period].iloc[0].yhat_lower

        # calculate percent changes based on predictions
        max_move = (future_close_max - self.datas[0].close[0])/self.datas[0].close[0]
        expected_move = (future_close_expected - self.datas[0].close[0])/self.datas[0].close[0]
        min_move = (future_close_min - self.datas[0].close[0])/self.datas[0].close[0]

        return (max_move, expected_move, min_move)    
        
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(FifthStrategy)

#4

df = select_stock('이노인스트루먼트','2019-01-01')
#df = df.iloc[:246]
df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
df = df.set_index(df['Date'])
dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

cerebro.adddata(dts)

#5
print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
cerebro.run()
print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

cerebro.plot(width=25,height=15)

In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt


class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.i = 0
        self.pastMA5 = 0
        self.pastMA20 = 0
        self.meanPrice = 0
        self.numberOfStocks = 0
    
    def notify_order(self, order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.meanPrice = (self.meanPrice * self.numberOfStocks + order.size * self.dataclose[0])/(self.numberOfStocks + order.size)
                self.numberOfStocks += order.size
            elif order.issell():
                self.meanPrice = 0
                self.numberOfStocks = 0
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        # print(str(self.datas[0].datetime.date(0)) + ' PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.i += 1
        if self.i < 20:
            return
        else:
            MA5 = np.mean([self.dataclose[n-4] for n in range(5)])
            MA20 = np.mean([self.dataclose[n-19] for n in range(20)])
            if MA5 > MA20 and self.pastMA5 < self.pastMA20 and self.broker.getcash() > self.dataclose[0]:
                amountToOrder = int(self.broker.getcash()/self.dataclose[0])
                self.order = self.buy(size=amountToOrder)
                print(self.i," - BUY : {}, buy amount : {}".format(self.dataclose[0], amountToOrder))
            
            elif self.meanPrice * 1.1 < self.dataclose[0] and self.numberOfStocks is not 0:
                print(self.i," - SELL : {}, sell amount : {}".format(self.dataclose[0], self.numberOfStocks))
                self.order = self.sell(size=self.numberOfStocks)

            self.pastMA5 = MA5
            self.pastMA20 = MA20
#1

#2
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(FirstStrategy)

#4

df = select_stock('삼성전자','2019-01-01')
df = df.iloc[:246]
df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
df = df.set_index(df['Date'])
dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

cerebro.adddata(dts)

#5
print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
cerebro.run()
print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

cerebro.plot()


In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt


class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.i = 0
        self.pastMA5 = 0
        self.pastMA20 = 0
        self.meanPrice = 0
        self.numberOfStocks = 0
    
    def notify_order(self, order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.meanPrice = (self.meanPrice * self.numberOfStocks + order.size * self.dataclose[0])/(self.numberOfStocks + order.size)
                self.numberOfStocks += order.size
            elif order.issell():
                self.meanPrice = 0
                self.numberOfStocks = 0
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        # print(str(self.datas[0].datetime.date(0)) + ' PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.i += 1
        if self.i < 20:
            return
        else:
            MA5 = np.mean([self.dataclose[n-4] for n in range(5)])
            MA20 = np.mean([self.dataclose[n-19] for n in range(20)])
            if MA5 > MA20 and self.pastMA5 < self.pastMA20 and self.broker.getcash() > self.dataclose[0]:
                amountToOrder = int(self.broker.getcash()/self.dataclose[0])
                self.order = self.buy(size=amountToOrder)
                print(self.i," - BUY : {}, buy amount : {}".format(self.dataclose[0], amountToOrder))
            
            elif self.meanPrice * 1.1 < self.dataclose[0] and self.numberOfStocks is not 0:
                print(self.i," - SELL : {}, sell amount : {}".format(self.dataclose[0], self.numberOfStocks))
                self.order = self.sell(size=self.numberOfStocks)

            self.pastMA5 = MA5
            self.pastMA20 = MA20
#1

#2
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(TestStrategy)

#4

df = select_stock('삼성전자','2019-01-01')
df = df.iloc[:246]
df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
df = df.set_index(df['Date'])
dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

cerebro.adddata(dts)

#5
print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
cerebro.run()
print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

cerebro.plot(width=25,height=15)



In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt


class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.i = 0
        self.pastMA5 = 0
        self.pastMA20 = 0
        self.meanPrice = 0
        self.numberOfStocks = 0
    
    def notify_order(self, order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.meanPrice = (self.meanPrice * self.numberOfStocks + order.size * self.dataclose[0])/(self.numberOfStocks + order.size)
                self.numberOfStocks += order.size
            elif order.issell():
                self.meanPrice = 0
                self.numberOfStocks = 0
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        # print(str(self.datas[0].datetime.date(0)) + ' PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.i += 1
        if self.i < 20:
            return
        else:
            MA5 = np.mean([self.dataclose[n-4] for n in range(5)])
            MA20 = np.mean([self.dataclose[n-19] for n in range(20)])
            if MA5 > MA20 and self.pastMA5 < self.pastMA20 and self.broker.getcash() > self.dataclose[0]:
                amountToOrder = int(self.broker.getcash()/self.dataclose[0])
                self.order = self.buy(size=amountToOrder)
                print(self.i," - BUY : {}, buy amount : {}".format(self.dataclose[0], amountToOrder))
            
            elif self.meanPrice * 1.1 < self.dataclose[0] and self.numberOfStocks is not 0:
                print(self.i," - SELL : {}, sell amount : {}".format(self.dataclose[0], self.numberOfStocks))
                self.order = self.sell(size=self.numberOfStocks)

            self.pastMA5 = MA5
            self.pastMA20 = MA20
#1

#2
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(TestStrategy)

#4

df = select_stock('삼성전자','2019-01-01')
df = df.iloc[:246]
df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
df = df.set_index(df['Date'])
dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

cerebro.adddata(dts)

#5
print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
cerebro.run()
print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

cerebro.plot()

In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt


class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.i = 0
        self.pastMA5 = 0
        self.pastMA20 = 0
        self.meanPrice = 0
        self.numberOfStocks = 0
    
    def notify_order(self, order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.meanPrice = (self.meanPrice * self.numberOfStocks + order.size * self.dataclose[0])/(self.numberOfStocks + order.size)
                self.numberOfStocks += order.size
            elif order.issell():
                self.meanPrice = 0
                self.numberOfStocks = 0
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        # print(str(self.datas[0].datetime.date(0)) + ' PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.i += 1
        if self.i < 20:
            return
        else:
            MA5 = np.mean([self.dataclose[n-4] for n in range(5)])
            MA20 = np.mean([self.dataclose[n-19] for n in range(20)])
            if MA5 > MA20 and self.pastMA5 < self.pastMA20 and self.broker.getcash() > self.dataclose[0]:
                amountToOrder = int(self.broker.getcash()/self.dataclose[0])
                self.order = self.buy(size=amountToOrder)
                print(self.i," - BUY : {}, buy amount : {}".format(self.dataclose[0], amountToOrder))
            
            elif self.meanPrice * 1.1 < self.dataclose[0] and self.numberOfStocks is not 0:
                print(self.i," - SELL : {}, sell amount : {}".format(self.dataclose[0], self.numberOfStocks))
                self.order = self.sell(size=self.numberOfStocks)

            self.pastMA5 = MA5
            self.pastMA20 = MA20
#1

#2
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(TestStrategy)

#4

df = select_stock('삼성전자','2019-01-01')
df = df.iloc[:246]
df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
df = df.set_index(df['Date'])
dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

cerebro.adddata(dts)

#5
print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
cerebro.run()
print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

cerebro.plot()

In [ ]:
df

In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt


class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.i = 0
        self.pastMA5 = 0
        self.pastMA20 = 0
        self.meanPrice = 0
        self.numberOfStocks = 0
    
    def notify_order(self, order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.meanPrice = (self.meanPrice * self.numberOfStocks + order.size * self.dataclose[0])/(self.numberOfStocks + order.size)
                self.numberOfStocks += order.size
            elif order.issell():
                self.meanPrice = 0
                self.numberOfStocks = 0
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        # print(str(self.datas[0].datetime.date(0)) + ' PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.i += 1
        if self.i < 20:
            return
        else:
            MA5 = np.mean([self.dataclose[n-4] for n in range(5)])
            MA20 = np.mean([self.dataclose[n-19] for n in range(20)])
            if MA5 > MA20 and self.pastMA5 < self.pastMA20 and self.broker.getcash() > self.dataclose[0]:
                amountToOrder = int(self.broker.getcash()/self.dataclose[0])
                self.order = self.buy(size=amountToOrder)
                print(self.i," - BUY : {}, buy amount : {}".format(self.dataclose[0], amountToOrder))
            
            elif self.meanPrice * 1.1 < self.dataclose[0] and self.numberOfStocks is not 0:
                print(self.i," - SELL : {}, sell amount : {}".format(self.dataclose[0], self.numberOfStocks))
                self.order = self.sell(size=self.numberOfStocks)

            self.pastMA5 = MA5
            self.pastMA20 = MA20
#1

#2
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

#3
cerebro.addstrategy(TestStrategy)

#4
for i in ['hrs','삼성전자']:
    df = select_stock(i,'2019-01-01')
    df = df.iloc[:246]
    df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
    df = df.set_index(df['Date'])
    dts = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

    cerebro.adddata(dts)

    #5
    print('Starting portfolio Value : {}'.format(cerebro.broker.getvalue()))
    cerebro.run()
    print('Final portfolio value : {}'.format(cerebro.broker.getvalue()))

    cerebro.plot()


In [ ]:
name=['hrs','오공']
    df = select_stock('삼성전자','2019-01-01')
    df = df.iloc[:246]
for idx, ticker in enumerate(['005930', '017670']):
    dataframe_collection.append(pd.read_sql("SELECT * FROM '{0}'".format(value), con, index_col='Date'))
    dataframe_collection[idx].index = pd.to_datetime( dataframe_collection[idx].index)
    data = bt.feeds.PandasData(dataname=dataframe_collection[idx])
    cerebro.adddata(data, name=ticker)


In [ ]:
for idx, ticker in enumerate(['005930', '017670']):
    print(idx)
    print(ticker)

In [ ]:
df = select_stock('삼성전자','2019-01-01')

In [ ]:
def abcd(trend, skip_loop = 4, ma = 7):
    ma = pd.Series(trend).rolling(ma).mean().values
    x = []
    for a in range(ma.shape[0]):
        for b in range(a, ma.shape[0], skip_loop):
            for c in range(b, ma.shape[0], skip_loop):
                for d in range(c, ma.shape[0], skip_loop):
                    if ma[b] > ma[a] and \
                    (ma[c] < ma[b] and ma[c] > ma[a]) \
                    and ma[d] > ma[b]:
                        x.append([a,b,c,d])
    x_np = np.array(x)
    ac = x_np[:,0].tolist() + x_np[:,2].tolist()
    bd = x_np[:,1].tolist() + x_np[:,3].tolist()
    ac_set = set(ac)
    bd_set = set(bd)
    signal = np.zeros(len(trend))
    buy = list(ac_set - bd_set)
    sell = list(list(bd_set - ac_set))
    signal[buy] = 1.0
    signal[sell] = -1.0
    return signal

In [ ]:
%%time
signal = abcd(df['Close'])

In [ ]:
def buy_stock(
    real_movement,
    signal,
    initial_money = 1000000,
    max_buy = 1,
    max_sell = 1,
):
    """
    real_movement = actual movement in the real world
    delay = how much interval you want to delay to change our decision from buy to sell, vice versa
    initial_state = 1 is buy, 0 is sell
    initial_money = 10000, ignore what kind of currency
    max_buy = max quantity for share to buy
    max_sell = max quantity for share to sell
    """
    starting_money = initial_money
    states_sell = []
    states_buy = []
    states_money = []
    current_inventory = 0
    
    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        if shares < 1:
            print(
                'day %d: total balances %f, not enough money to buy a unit price %f'
                % (i, initial_money, real_movement[i])
            )
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(
                'day %d: buy %d units at price %f, total balance %f'
                % (i, buy_units, buy_units * real_movement[i], initial_money)
            )
            states_buy.append(0)
        return initial_money, current_inventory
    
    for i in range(real_movement.shape[0]):
        state = signal[i]
        if state == 1:
            initial_money, current_inventory = buy(
                i, initial_money, current_inventory
            )
            states_buy.append(i)
        elif state == -1:
            if current_inventory == 0:
                    print('day %d: cannot sell anything, inventory 0' % (i))
            else:
                if current_inventory > max_sell:
                    sell_units = max_sell
                else:
                    sell_units = current_inventory
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                try:
                    invest = (
                        (real_movement[i] - real_movement[states_buy[-1]])
                        / real_movement[states_buy[-1]]
                    ) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (i, sell_units, total_sell, invest, initial_money)
                )
            states_sell.append(i)
        states_money.append(initial_money)
            
    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest, states_money

In [ ]:
states_buy, states_sell, total_gains, invest, states_money = buy_stock(df.Close, signal)

In [ ]:
fig = plt.figure(figsize = (15,5))
plt.plot(close, color='r', lw=2.)
plt.plot(close, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
plt.plot(close, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
plt.title('total gains %f, total investment %f%%'%(total_gains, invest))
plt.legend()
plt.show()

In [ ]:
df.loc[' 2020-01'].head()

# Backtrader 

## 주식Data를 select_stock로 불러서 백테스트할때

In [ ]:
##  20일 이동 평균선 추세매매 (20MA > Close : Buy,  20MA < Close : Sell)

from mod1 import *
import datetime
import backtrader as bt

# Create a subclass of Strategy to define the indicators and logic
class Momentum(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=20  # period for the slow moving average
    )

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.smaSlow = bt.ind.SimpleMovingAverage(period=self.p.pslow)
        self.smaFast = bt.ind.SimpleMovingAverage(period=self.p.pfast)
        self.order = None

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm: {3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm{3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self)  # when was trade executed
        # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        cash = self.broker.get_cash()
        value = self.broker.get_value()
        size = int(cash / self.data.close[0])
        # Order가 Pending인지 확인, 그렇다면 다시 주문할 수 없음
        if self.order:
            return

        if not self.position:  # not in the market
            if self.smaSlow < self.data.close[0]:
                self.order = self.buy(size=size)

        elif self.getposition().size > 0: #in the market
            if self.smaSlow > self.data.close[0]:
                self.order = self.sell(size=self.getposition().size)

def run(args=None):
    cerebro = bt.Cerebro()  # create a "Cerebro" engine instance
    cerebro.broker.setcash(1000000) #초기자금

    df = select_stock('삼성전자','2019-01-01')
    df = df.iloc[:246]
    df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
    df = df.set_index(df['Date'])
    data = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

    cerebro.adddata(data) #데이터 삽입
    cerebro.addstrategy(Momentum) #전략적용
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run() #수행
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot() #plot

if __name__ == '__main__':
    run()

In [ ]:
##  20일 이동 평균선 역추세매매 (20MA > Close : Sell,  20MA < Close : Buy)

from mod1 import *
import datetime
import backtrader as bt

# Create a subclass of Strategy to define the indicators and logic
class Momentum(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=20  # period for the slow moving average
    )

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.smaSlow = bt.ind.SimpleMovingAverage(period=self.p.pslow)
        self.smaFast = bt.ind.SimpleMovingAverage(period=self.p.pfast)
        self.order = None

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm: {3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm{3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self)  # when was trade executed
        # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        cash = self.broker.get_cash()
        value = self.broker.get_value()
        size = int(cash / self.data.close[0])
        # Order가 Pending인지 확인, 그렇다면 다시 주문할 수 없음
        if self.order:
            return

        if not self.position:  # not in the market
            if self.smaSlow > self.data.close[0]:
                self.order = self.buy(size=size)

        elif self.getposition().size > 0: #in the market
            if self.smaSlow < self.data.close[0]:
                self.order = self.sell(size=self.getposition().size)
                
def run(args=None):
    cerebro = bt.Cerebro()  # create a "Cerebro" engine instance
    cerebro.broker.setcash(1000000) #초기자금

    df = select_stock('삼성전자','2019-01-01')
    df = df.iloc[:246]
    df["Date"] = pd.to_datetime(df["Date"])  ###  아주 중요!! pandas.core.series.Series DATA를 datatime object numpy로 돌려줌
    df = df.set_index(df['Date'])
    data = bt.feeds.PandasData(dataname=df, open='Open',close='Close')

    cerebro.adddata(data) #데이터 삽입
    cerebro.addstrategy(Momentum) #전략적용
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run() #수행
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot() #plot

if __name__ == '__main__':
    run()

## 주식Data를 text로 불러서 백테스트할때

In [ ]:
import datetime
import backtrader as bt


# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed
#homepath = os.getenv('c:\\Users\\kkang\notebook\\Git\\first_module')
datapath = ('./orcl-1995-2014.txt')

data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    fromdate=datetime.datetime(2000,1,1),
    todate = datetime.datetime(2000,12,31),
    reverse=False)

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy
cerebro.run()  # run it all
cerebro.plot(style='candlestick')  # and plot it with a single command

In [ ]:
type(data)

In [ ]:
##  기간동안 낙폭 과대종목 검색_1

import time
from  mod1 import *

start = time.time()  # 시작 시간 저장

def all_stock_period(date1, date2='2021-01-01'):
    select_query = "select * from market_good where Date >=  "
    var = select_query +"'"+date1+"'"  +" "+ 'and Date <=' + "'"+date2+"'"
    df = pd.read_sql(var, engine)
    return df


df = all_stock_period('2020-02-20','2020-03-25')
df_uniq = df['Name'].unique()
df_uniq_list=df_uniq.tolist()

min_data = []
for x in df_uniq_list:
    min_value = min(df[df['Name']== x ].Close)
    min_data.append(min_value)

min_close = pd.DataFrame(min_data)

df_a=pd.DataFrame(df_uniq)


df_first=pd.DataFrame()
df_first['Name']=df_a[0]
df_first['Close']=min_close[0]
df_to = all_stock('2020-08-18')
df_last=df_to[['Name','Close']]
df = pd.merge(df_first,df_last,on='Name')

df['diff']=df['Close_y']/df['Close_x']
df.head()

close_diff_df =  df.sort_values(["diff"],ascending=True)
close_diff_df.head()

close_diff_df.to_excel("f:\\b_9.xlsx")
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
##  기간동안 낙폭 과대종목 검색 (기간동안 최저점에서 현재가 비교)_2

import time
from  mod1 import *

df_date = pd.read_sql("select Date from market where Name='hrs' order by Date desc limit 1",engine)
df_date = pd.to_datetime(df_date['Date'])
#df = df + timedelta(1)          ##  최종날짜 다음날짜
df_date = str(df_date)
standard = df_date[4:14]                ## 2020-07-13

def all_stock_period(date1, date2='2021-01-01'):
    select_query = "select * from market_good where Date >=  "
    var = select_query +"'"+date1+"'"  +" "+ 'and Date <=' + "'"+date2+"'"
    df = pd.read_sql(var, engine)
    return df

def flow_close(method, standard_day=standard):
    start = time.time()  # 시작 시간 저장

    df = method
    #df_start=pd.DataFrame()
    name = df['Name'].unique()

    df = df.set_index('Name')

    min_list=[ ]
    for i in name:
        try:
            #min_close  = min([df.loc[i].Close])
            min_close  = df.loc[i].Close.min()
            #print('minum-', min_close)
            #min_list.extend(str(min(df.loc[i].Close)))
            min_list.append(min_close)
        except:
            min_close  = df.loc[i].Close
            min_list.append(min_close)
    df_start=pd.DataFrame(name)
    df_start.rename(columns={0:'Name'}, inplace=True)
    df_start['Close']=pd.DataFrame(min_list)
    #df_start=df_start[['Name','Close']]
    df1 = all_stock_at(standard_day)
    df1_last = df1[['Name','Close']]

    df2 = pd.merge(df_start, df1_last, on="Name")
    df2['diff']=df2['Close_y']/df2['Close_x']
    df2 = df2.sort_values(by=['diff'], ascending='True')

    df2.to_excel('d:\\down_'+standard_day+'.xlsx')
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
flow_close(all_stock_period('2020-02-19','2020-03-25'))
#flow_close(all_stock_period('2010-03-01','2020-03-31'))
#flow_close(all_stock_period('2009-01-01','2020-03-31'))    

In [ ]:
##  기간동안 낙폭 과대종목 검색 (기간동안 최저점에서 현재가 비교)_3  ###  일별로 각각 비교

from  mod1 import *

start = time.time()

df_date = pd.read_sql("select Date from market where Name='hrs' order by Date desc limit 1",engine)
df_date = pd.to_datetime(df_date['Date'])
#df = df + timedelta(1)          ##  최종날짜 다음날짜
df_date = str(df_date)
standard = df_date[4:14]                ## 2020-07-13

def flow(from_day, to_day,  standard_day=standard):
    start = time.time()  # 시작 시간 저장

    df = all_stock_certain_period(from_day, to_day)
    #df_start=pd.DataFrame()
    name = df['Name'].unique()

    #df = df.set_index('Name')
    return df

df = flow('2020-02-19', '2020-03-25', standard)

name = df['Name'].unique()
df = df.set_index('Name')

min_list=[]
for i in name:
    try:
        min_close  = df.loc[i].Close.min()
        min_list.append(min_close)
    except:
        pass

df_start=pd.DataFrame(name)

df_start.rename(columns={0:'Name'}, inplace=True)
df_start['Close']=pd.DataFrame(min_list)

standard_to_datetime = pd.to_datetime(standard)
for i in range(2):
    daily = standard_to_datetime + timedelta(-i)
    daily_str = str(daily)
    daily_str = daily_str[:11]

    df1 = all_stock_at( daily_str)
    df1_last = df1[['Name','Close']]

    df2 = pd.merge(df_start, df1_last, on="Name")

    df2['diff']=df2['Close_y']/df2['Close_x']
    df2 = df2.sort_values(by=['diff'], ascending='True')

    df2.to_excel('d:\\down_'+daily_str+'.xlsx') 
print('time : ' , time.time()-start)


In [ ]:
## 개별종목분석

from mod1 import *

def close_ma_vol(df,select1,select2,select3):
    ma(df)

    source = MinMaxScaler()
    data = source.fit_transform(df[['close',select1,select2,select3]].values)
    df1 = pd.DataFrame(data)
    df1.columns=['close',select1,select2,select3]
    df1 = df1.set_index(df['date'])
    df1.plot(figsize=(16,4))
    plt.title(df['name'][0])
    plt.grid(True)
    plt.show()    

name=['디엔에프','모트렉스','푸드나무','상보','아난티','손오공','한컴위드']

for i in name:
    df = select_stock(i,'2018-01-01')
    close_ma_vol(df,'ma60','ma120','volume')

In [ ]:
##  거래량 증가 관심종목 분석
from mod1 import *

def volume_group_analysis(path_volume,select_day):
    select_day = '2020-04-27'
    #date = '2020-01-01'
    name = pd.read_excel(path_volume+select_day+'.xlsx', encoding='utf-8',index_col=0)
    name_all = name['Name']
    name_all = name_all.to_list()
    name = name[:5]

    for i in name_all:
        select_query = "select Date,Volume,Close from market where Name= "
        date_query = "Date > "
        #var = select_query +"'"+i+"'"+" "+"&&"+" "+date_query+"'"+date+"'" 
        var = select_query +"'"+i+"'"+" "+"&&"+" "+date_query+"'"+select_day+"'" 
        df = pd.read_sql(var, engine)
        df[['Volume','Close']] = df[['Volume','Close']].astype(float) #  TA-Lib로 평균을 구하려면 실수로 만들어야 함
        df.columns=df.columns.str.lower()

        talib_ma = ta.MA(df, timeperiod=20)
        df['ma20'] = talib_ma

        source = MinMaxScaler()
        data = source.fit_transform(df[['close','volume','ma20']].values)
        df1 = pd.DataFrame(data)
        df1.columns=['close','volume','ma20']
        df1 = df1.set_index(df['date'])
        df1.plot(figsize=(16,2))
        plt.title(i)
        plt.show()
               
volume_group_analysis(path_volume,'2020-04-27')

In [ ]:
##  가격 증가 관심종목 분석

from mod1 import *

def price_group_analysis(path_price,select_day):
    #select_day = '2020-04-27'
    #date = '2020-01-01'
    name = pd.read_excel(path_volume+select_day+'.xlsx', encoding='utf-8',index_col=0)
    name_all = name['Name']
    name_all = name_all.to_list()
    name = name[:5]

    for i in name_all:
        select_query = "select Date,Volume,Close from market where Name= "
        date_query = "Date > "
        var = select_query +"'"+i+"'"+" "+"&&"+" "+date_query+"'"+select_day+"'" 
        df = pd.read_sql(var, engine)
        df[['Volume','Close']] = df[['Volume','Close']].astype(float) #  TA-Lib로 평균을 구하려면 실수로 만들어야 함
        df.columns=df.columns.str.lower()

        talib_ma120 = ta.MA(df, timeperiod=20)
        df['ma20'] = talib_ma120

        source = MinMaxScaler()
        data = source.fit_transform(df[['close','volume','ma20']].values)
        df1 = pd.DataFrame(data)
        df1.columns=['close','volume','ma20']
        df1 = df1.set_index(df['date'])
        df1.plot(figsize=(16,2))
        plt.title(i)
        plt.show()

price_group_analysis(path_price,'2020-05-27')

# stock_prediction tensorflow 1.0

# first

In [ ]:
###  Keras_LSTM_Stock_Prediction#1

from mod1 import *
import math
import tensorflow as tf

df = select_stock('hrs','2020-01-01')
df=df[['Open','High','Close']]

df['High'] = df['High'] / 100000
df['Open'] = df['Open'] / 100000
df['Close'] = df['Close'] / 100000
df.head(5)

window = 5
amount_of_features = len(df.columns)
data = df.values #pd.DataFrame(stock)
sequence_length = window + 1
result = []

for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])

result = np.array(result)



row = round(0.9 * result.shape[0])
train = result[:int(row), :]
x_train = train[:, :-1]
y_train = train[:, -1][:,-1]
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1][:,-1]

#x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("X_test", x_test.shape)
print("y_test", y_test.shape)


d = 0.2
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(window, 3), return_sequences=True))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.LSTM(64, input_shape=(window, 3), return_sequences=False))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.Dense(16,kernel_initializer="uniform",activation='relu'))        
model.add(tf.keras.layers.Dense(1,kernel_initializer="uniform",activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=10,epochs=100,validation_split=0.1,verbose=0)

trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

# print(X_test[-1])
diff=[]
ratio=[]
p = model.predict(x_test)
for u in range(len(y_test)):
    pr = p[u][0]
    ratio.append((y_test[u]/pr)-1)
    diff.append(abs(y_test[u]- pr))
    #print(u, y_test[u], pr, (y_test[u]/pr)-1, abs(y_test[u]- pr))

plt.plot(p,color='red', label='prediction')
plt.plot(y_test,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
###  Keras_LSTM_Stock_Prediction#2


from mod1 import *
import math
import tensorflow as tf

df = select_stock('hrs','2020-01-01')
df=df[['Open','High','Close']]

scaler = MinMaxScaler()

window = 5
amount_of_features = len(df.columns)
data = df.values #pd.DataFrame(stock)
data = scaler.fit_transform(data)
sequence_length = window + 1
result = []

for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])
                  
result = np.array(result)

row = round(0.9 * result.shape[0])
train = result[:int(row), :]

x_train = train[:, :-1]
y_train = train[:, -1][:,-1]
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1][:,-1]

print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

d = 0.2
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(window, 3), return_sequences=True))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.LSTM(64, input_shape=(window, 3), return_sequences=False))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.Dense(16,kernel_initializer="uniform",activation='relu'))        
model.add(tf.keras.layers.Dense(1,kernel_initializer="uniform",activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=10,epochs=500,verbose=0)
model.evaluate(x_train, y_train)

pred = model.predict(x_test) 
#pred = [[0.69005],[0.61538],[0.60407],[0.63575],[0.62443]]  ## 테스트용으로 종가 5일치를 역순으로 
scale= MinMaxScaler()
scale.min_, scale.scale_=scaler.min_[2], scaler.scale_[2]   ##  Open열[0],High열[1] ~ Close열은 [4]
pred = scale.inverse_transform(pred)
print(pred)

true = y_test.reshape(-1,1)
true = scale.inverse_transform(true)
print(true)

plt.plot(pred,color='red', label='prediction')
plt.plot(true,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

# Second

In [ ]:
## Model Save - MinMaxScaler.fit 

import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
sns.set()
tf.compat.v1.random.set_random_seed(1234)

tf.random.set_seed(777)  # reproducibility

df = select_stock('hrs','2020-01-01')
df = df[['Open','High','Low','Volume','Close']]
xy = df.values

test_min = np.min(xy, 0)
test_max = np.max(xy, 0)
test_denom = test_max - test_min


seq_length = 5
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 9000

minmax = MinMaxScaler().fit(df.astype('float64')) # Close index
dataset = minmax.transform(df.astype('float64')) # Close index

train_size = int(len(dataset) * 0.7)
train_set = dataset[0:train_size]
test_set = dataset[train_size-seq_length:]  # Index from [train_size - seq_leng

# build datasets
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]]  # Next close price
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print("trainX", trainX.shape)
print("trainY", trainX.shape)
print("testX", testX.shape)
print("testY", testY.shape)


d = 0.2
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(5, 5), return_sequences=True))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.LSTM(64, input_shape=(5, 5), return_sequences=False))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.Dense(16,kernel_initializer="uniform",activation='relu'))        
model.add(tf.keras.layers.Dense(1,kernel_initializer="uniform",activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.fit(trainX, trainY,batch_size=10,epochs=100,validation_split=0.1,verbose=0)

model.evaluate(trainX, trainY)

pred = model.predict(testX) 
#pred = [[0.69005],[0.61538],[0.60407],[0.63575],[0.62443]]  ## 테스트용으로 종가 5일치를 역순으로 
scale= MinMaxScaler()
scale.min_, scale.scale_=minmax.min_[4], minmax.scale_[4]   ##  Open열[0],High열[1] ~ Close열은 [4]
pred = scale.inverse_transform(pred)
print(pred)

true = testY.reshape(-1,1)
true = scale.inverse_transform(true)
print(true)

plt.plot(pred,color='red', label='prediction')
plt.plot(true,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
'''
This script shows how to predict stock prices using a basic RNN
'''
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm


tf.random.set_seed(777) 

df = select_stock('hrs','2020-01-01')
df = df[['Open','High','Low','Volume','Close']]
xy = df.values


# train Parameters
seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 1000

scaler = MinMaxScaler()
dataset = scaler.fit_transform(xy)

# train/test split
train_size = int(len(dataset) * 0.7)
pre_train_set = dataset[0:train_size]
pre_test_set = dataset[train_size - seq_length:]  # Index from [train_size - seq_length] to utilize past sequence

#최종 7 rows 다음날 Price Predict 입력 data
last_X = scaler.fit_transform((xy[-seq_length:,:]))


# build datasets
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]]  # Next close price
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainx, trainy = build_dataset(pre_train_set, seq_length)
print("#"*100)
testx, testy = build_dataset(pre_test_set, seq_length)

d = 0.2
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(5, 5), return_sequences=True))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.LSTM(64, input_shape=(5, 5), return_sequences=False))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.Dense(16,kernel_initializer="uniform",activation='relu'))        
model.add(tf.keras.layers.Dense(1,kernel_initializer="uniform",activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.fit(trainX, trainY,batch_size=10,epochs=100,validation_split=0.1,verbose=0)

trainScore = model.evaluate(trainX, trainY, verbose=1)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(testX, testY, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

# print(X_test[-1])
diff=[]
ratio=[]
predict=[]
p = model.predict(testX)
for u in range(len(testY)):
    pr = p[u][0]
    ratio.append((testY[u]/pr)-1)
    diff.append(abs(testY[u]- pr))
    predict.append(pr)
    #print(u, y_test[u], pr, (y_test[u]/pr)-1, abs(y_test[u]- pr))

plt.plot(p,color='red', label='prediction')
plt.plot(testY,color='blue', label='testY')
plt.legend(loc='upper left')
plt.show()

# Third

In [ ]:
from mod1 import *
import tensorflow as tf

df = select_stock('hrs','2010-01-01')
# Line 9 to 17 is for preprocessing and saving the dataset downloaded from yahoo website
# It is not necessary to run these codes if you are using provided dataset in repo.
# If you want to use your own dataset downloaded from yahoo then first run these with commented rest.
# After that you can run as usual provided.

# df = df.drop(['Date'],axis=1)
# for col in df:
#     for i,item in enumerate(df[col]):
#         if item=='null':
#             df[col][i] = np.nan
# df = df.dropna(inplace=False)
# for col in df:
#     print(df[col].isnull().sum())
# df.to_csv('yah.csv',index=False)

df = df.drop(['Date','Code','Name','Volume'],axis=1)


df_train = df[:1059]
df_test = df[1059:]
scaler = MinMaxScaler()

X_train = scaler.fit_transform(df_train.drop(['Close'],axis=1).values)
y_train = scaler.fit_transform(df_train['Close'].values.reshape(-1, 1))


X_test = scaler.fit_transform(df_test.drop(['Close'],axis=1).values)
y_test = scaler.fit_transform(df_test['Close'].values.reshape(-1, 1))
#y_test = df_test['Close'].as_matrix()
print(X_train.shape)
print(np.max(y_test),np.max(y_train),np.min(y_test),np.min(y_train))

def denormalize(df,norm_data):
    df = df['Close'].values.reshape(-1,1)
    norm_data = norm_data.reshape(-1,1)
    scl = MinMaxScaler()
    a = scl.fit_transform(df)
    new = scl.inverse_transform(norm_data)
    return new

def neural_net_model(X_data,input_dim):
    W_1 = tf.Variable(tf.random_uniform([input_dim,10]))
    b_1 = tf.Variable(tf.zeros([10]))
    layer_1 = tf.add(tf.matmul(X_data,W_1), b_1)
    layer_1 = tf.nn.tanh(layer_1)

    W_2 = tf.Variable(tf.random_uniform([10,10]))
    b_2 = tf.Variable(tf.zeros([10]))
    layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
    layer_2 = tf.nn.tanh(layer_2)

    W_O = tf.Variable(tf.random_uniform([10,1]))
    b_O = tf.Variable(tf.zeros([1]))
    output = tf.add(tf.matmul(layer_2,W_O), b_O)

    return output,W_O

In [ ]:
y_t = denormalize(df_train,y_train)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.xlabel('Days')
plt.ylabel('Stock Value HRS')
plt.title('HRS Stock Index Prediction')
ax.plot(range(len(y_train)), y_t,label='Original')
plt.ion()


d = 0.2
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(1, 3), return_sequences=True))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.LSTM(64, input_shape=(1, 3), return_sequences=False))
model.add(tf.keras.layers.Dropout(d))
model.add(tf.keras.layers.Dense(16,kernel_initializer="uniform",activation='relu'))        
model.add(tf.keras.layers.Dense(1,kernel_initializer="uniform",activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.fit(X_train, y_train,batch_size=10,epochs=100,validation_split=0.1,verbose=0)


In [ ]:
X_train.ndim

In [ ]:
df['Close'].values.reshape(-1,1)

In [ ]:
def aaa(norm_data):
    norm_data = norm_data.reshape(-1,1)
    scl = MinMaxScaler()
    new = scl.inverse_transform(norm_data)
    return new

In [ ]:
##  Tensorflow 1.0

from mod1 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior( )

df = select_stock('hrs','2010-01-01')
# Line 9 to 17 is for preprocessing and saving the dataset downloaded from yahoo website
# It is not necessary to run these codes if you are using provided dataset in repo.
# If you want to use your own dataset downloaded from yahoo then first run these with commented rest.
# After that you can run as usual provided.

# df = df.drop(['Date'],axis=1)
# for col in df:
#     for i,item in enumerate(df[col]):
#         if item=='null':
#             df[col][i] = np.nan
# df = df.dropna(inplace=False)
# for col in df:
#     print(df[col].isnull().sum())
# df.to_csv('yah.csv',index=False)

df = df.drop(['Date','Code','Name','Volume'],axis=1)


df_train = df[:1059]
df_test = df[1059:]
scaler = MinMaxScaler()

X_train = scaler.fit_transform(df_train.drop(['Close'],axis=1).values)
y_train = scaler.fit_transform(df_train['Close'].values.reshape(-1, 1))


X_test = scaler.fit_transform(df_test.drop(['Close'],axis=1).values)
y_test = scaler.fit_transform(df_test['Close'].values.reshape(-1, 1))
#y_test = df_test['Close'].as_matrix()
print(X_train.shape)
print(np.max(y_test),np.max(y_train),np.min(y_test),np.min(y_train))


def denormalize(df,norm_data):
    df = df['Close'].values.reshape(-1,1)
    norm_data = norm_data.reshape(-1,1)
    scl = MinMaxScaler()
    a = scl.fit_transform(df)
    new = scl.inverse_transform(norm_data)
    return new

def neural_net_model(X_data,input_dim):
    W_1 = tf.Variable(tf.random_uniform([input_dim,10]))
    b_1 = tf.Variable(tf.zeros([10]))
    layer_1 = tf.add(tf.matmul(X_data,W_1), b_1)
    layer_1 = tf.nn.tanh(layer_1)

    W_2 = tf.Variable(tf.random_uniform([10,10]))
    b_2 = tf.Variable(tf.zeros([10]))
    layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
    layer_2 = tf.nn.tanh(layer_2)

    W_O = tf.Variable(tf.random_uniform([10,1]))
    b_O = tf.Variable(tf.zeros([1]))
    output = tf.add(tf.matmul(layer_2,W_O), b_O)

    return output,W_O

xs = tf.placeholder("float")
ys = tf.placeholder("float")

output,W_O = neural_net_model(xs,3)

cost = tf.reduce_mean(tf.square(output-ys))
train = tf.train.AdamOptimizer(0.001).minimize(cost)

correct_pred = tf.argmax(output, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

c_t = []
c_test = []


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()
    y_t = denormalize(df_train,y_train)
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.xlabel('Days')
    plt.ylabel('Stock Value HRS')
    plt.title('HRS Stock Index Prediction')
    ax.plot(range(len(y_train)), y_t,label='Original')
    plt.ion()

    #saver.restore(sess,'yahoo_dataset.ckpt')
    for i in range(100):
        #sess.run([cost,train],feed_dict={xs:X_train, ys:y_train})
        for j in range(X_train.shape[0]):
            sess.run([cost,train],feed_dict={xs:X_train[j,:].reshape(1,3), ys:y_train[j]})

        try:
            ax.lines.remove(lines[0])
        except Exception:
            pass
        pred = sess.run(output, feed_dict={xs:X_train})
        pred = denormalize(df_train,pred)
        plt.plot(range(len(y_train)), pred,'r-',label='Prediction')
        plt.legend(loc='best')
        plt.pause(0.1)

        c_t.append(sess.run(cost, feed_dict={xs:X_train,ys:y_train}))
        c_test.append(sess.run(cost, feed_dict={xs:X_test,ys:y_test}))
        print('Epoch :',i,'Cost :',c_t[i])

    pred = sess.run(output, feed_dict={xs:X_test})
    for i in range(y_test.shape[0]):
        print('Original :',y_test[i],'Predicted :',pred[i])

    #plt.plot(range(50),c_t)
    #plt.plot(range(50),c_test)
    #plt.show()

    print('Cost :',sess.run(cost, feed_dict={xs:X_test,ys:y_test}))
    y_test = denormalize(df_test,y_test)
    pred = denormalize(df_test,pred)
    plt.plot(range(y_test.shape[0]),y_test,label="Original Data")
    plt.plot(range(y_test.shape[0]),pred,label="Predicted Data")
    plt.legend(loc='best')
    """plt.ylabel('Stock Value')
    plt.xlabel('Days')
    plt.title('Stock Market Nifty')"""
    plt.show()
    if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() + '/yahoo_dataset.ckpt')
        print('Model Saved')

# tensorflow 2.0

In [ ]:
## LSTM stock_prediction without MinMax()

from mod1 import *
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

#1. Data 전처리

df = select_stock('hrs','2020-09-07')
df=df[['Date',"Close"]]

df = df.set_index('Date')
df1 = df.values


x, y = list(),list()
x_end = 5
y_end = x_end + 3

for i in range(df.shape[0]-y_end+1):
    x_tmp=df1[i:i+x_end:]
    y_tmp=df1[i+x_end:i+y_end,0] 
    x.append(x_tmp)
    y.append(y_tmp)

x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
print(x_train.shape)
print(x_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)
print(x_train_scaled[0, :])

x_train_scaled = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 5, 1))
x_test_scaled = np.reshape(x_test_scaled, (x_test_scaled.shape[0], 5, 1))
print(x_train_scaled.shape)
print(x_test_scaled.shape)

from keras.models import Sequential
from keras.layers import Dense, LSTM,Dropout

# 모델구성
model = Sequential()
model.add(LSTM(64, input_shape=(5, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3))

# train
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)
model.fit(x_train_scaled, y_train, validation_split=0.2, verbose=1, batch_size=1, epochs=100, callbacks=[early_stopping])

loss, mse = model.evaluate(x_test_scaled, y_test, batch_size=1)
print('loss : ', loss)
print('mse : ', mse)

# predict
y_pred = model.predict(x_test_scaled)

for i in range(5):
    print('종가 : ', y_test[i], '/ 예측가 : ', y_pred[i])

# Super Trend

In [ ]:
from mod1 import *

df = select_stock('한국알콜','2020-01-01')

# -*- coding: utf-8 -*-
"""
@author: techietrader
"""

import numpy as np
import pandas as pd  

def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.at[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas version- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return df
     


# -*- coding: utf-8 -*-
"""
@author: techietrader
"""

import numpy as np
import pandas as pd  


#SuperTrend
def ST(df,f,n): #df is the dataframe, n is the period, f is the factor; f=3, n=7 are commonly used.
    #Calculation of ATR
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.at[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas verion- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n

    #Calculation of SuperTrend
    df['Upper Basic']=(df['High']+df['Low'])/2+(f*df['ATR'])
    df['Lower Basic']=(df['High']+df['Low'])/2-(f*df['ATR'])
    df['Upper Band']=df['Upper Basic']
    df['Lower Band']=df['Lower Basic']
    for i in range(n,len(df)):
        if df['Close'][i-1]<=df['Upper Band'][i-1]:
            df['Upper Band'][i]=min(df['Upper Basic'][i],df['Upper Band'][i-1])
        else:
            df['Upper Band'][i]=df['Upper Basic'][i]    
    for i in range(n,len(df)):
        if df['Close'][i-1]>=df['Lower Band'][i-1]:
            df['Lower Band'][i]=max(df['Lower Basic'][i],df['Lower Band'][i-1])
        else:
            df['Lower Band'][i]=df['Lower Basic'][i]   
    df['SuperTrend']=np.nan
    for i in df['SuperTrend']:
        if df['Close'][n-1]<=df['Upper Band'][n-1]:
            df['SuperTrend'][n-1]=df['Upper Band'][n-1]
        elif df['Close'][n-1]>df['Upper Band'][i]:
            df['SuperTrend'][n-1]=df['Lower Band'][n-1]
    for i in range(n,len(df)):
        if df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close'][i]<=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
        elif  df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close'][i]>=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close'][i]>=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close'][i]<=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
    return df
    


In [ ]:
def SuperST(stockdata,estim,multiplier = 7):
    #first we need next row value
    df_prev = estim[['finaluband','finallband','supert']].join(stockdata[['close']]).shift(1)
    # name colums as it was in paramas
    df_prev.columns = ['prevfinaluband', 'prevfinallband', 'prevsupertrend', 'prevclose']
    # working matrix parmas with shifed values w/o NaN in ATR
    df = stockdata.dropna().join(df_prev).fillna(0)

    #calulate vectors 
    upperbasicband = ((df.high+df.low)/2 + (multiplier * df.atr))
    lowerbasicband = ((df.high+df.low)/2 - (multiplier * df.atr))
    # np.where is a vector version of if
    upperband = np.where((upperbasicband < df.prevfinaluband) | (df.prevclose > df.prevfinaluband),
                        upperbasicband,df.prevfinaluband)
    lowerband = np.where ((lowerbasicband > df.prevfinallband) | (df.prevclose < df.prevfinallband),
                         lowerbasicband,df.prevfinallband)

    supertrend = np.where (df.prevsupertrend == df.prevfinaluband,
                             np.where(df.close <= upperband,
                                      upperband,lowerband),
                          np.where(df.prevsupertrend == df.prevfinallband,
                                      np.where(df.close >= lowerband,
                                              lowerband,upperband),
                                    np.zeros_like(lowerband)))


    return upperbasicband.to_frame('uband').assign( 
                    lband = lowerbasicband).assign(
                    finaluband = upperband).assign(
                    finallband = lowerband).assign(
                    supert = supertrend)

In [ ]:
#df = pd.read_excel('d:\\stockdata\\vote_stock\\detect_stock_with_price_2020-05-18.xlsx')
df = pd.read_excel('d:\\b_9.xlsx')
df=df['Name'].iloc[300:400]
#name = df.values.tolist() ## numpy to list
name = df.to_list()  

#name = ['SGA솔루션즈','한국전자인증','SGA','케이사인','한컴위드']

for i in name:
    df = select_stock(i,'2020-01-01')
    ST(df,3,7)
    df = df.set_index(df['Date'])
    df[['Close','SuperTrend']].plot(figsize=(16,4))
    plt.title(i)
    plt.grid(True)
    plt.show()    


In [ ]:
name = '원익홀딩스'
df = select_stock(name,'2017-01-01')
ST(df,3,7)
df = df.set_index(df['Date'])
df[['Close','SuperTrend']].plot(figsize=(16,4))
plt.title(name)
plt.grid(True)
plt.show()    


In [ ]:
#df = pd.read_excel('d:\\stockdata\\vote_stock\\detect_stock_with_price_2020-05-18.xlsx')
#df = pd.read_excel('d:\\high.xlsx')
#df = df['Name'].unique()
#name = df.values.tolist() ## numpy to list
#name = df.tolist()  

name = ['모트렉스','원익홀딩스','오공','동국산업','링네트','이씨에스','한국단자']

for i in name:
    df = select_stock(i,'2019-01-01')
    df['V']=df['Volume']
    ST(df,3,7)
    df = df.set_index(df['Date'])
    df[['Close','SuperTrend']].plot(figsize=(16,4))
    plt.title(i)
    plt.grid(True)
    plt.show()    



In [ ]:
df = select_stock('hrs','2020-05-01')
df

In [ ]:
var = "select * from kospi where Date > '2018-01-01'"
df = pd.read_sql(var, engine)


ST(df,3,7)
df = df.set_index(df['Date'])
df[['Close','SuperTrend']].plot(figsize=(16,4))
plt.title('kospi')
plt.grid(True)
plt.show()

In [ ]:
var = "select * from kospi where Date > '2019-06-01'"
df = pd.read_sql(var, engine)


ST(df,3,7)
df = df.set_index(df['Date'])
df[['Close','SuperTrend']].plot(figsize=(16,4))
plt.title('kospi')
plt.grid(True)
plt.show()

In [ ]:
from mod1 import *
import matplotlib.gridspec as gridspec
import mplfinance as mpf


df = select_stock('hrs', '2020-01-01')

ohlc = df[['Date','Open', 'High', 'Low', 'Close','Volume']]
ohlc['Date'] = pd.to_datetime(ohlc['Date'])
ohlc = ohlc.set_index('Date')
ohlc.index.name = 'Date'

mpf.plot(ohlc,figratio=(15,8),type='candle',mav=(5),volume=True,style='charles')  ### 3,6,9일 이동평균선 그리고 거래량

# 데이타 전처리

## MinMaxScaler

In [ ]:
from mod1 import *

df = select_stock('hrs','2020-01-01')
df = df[['Open','High','Low','Volume','Close']]
print(df.head())

# 결과확인을 위해 소수 출력 옵션 변경 소수 첫째자리까지
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)}) 

data = df.values

scaler = MinMaxScaler()

data_a =scaler.fit_transform(data)               ## 전처리 
print(data_a[:5])

data_b = scaler.inverse_transform(data_a)        ## 전체 복원
print(data_b[:5])

data_c = scaler.inverse_transform(data_a)[:, [0]] ##  Open열만 복원,  Close열은 [4]
data_c[:5]

### 주식 예측값 배열이 생성 되었을때 실제 Close와 비교하기

#pred = model.predict(test_X) 
pred = [[0.69005],[0.61538],[0.60407],[0.63575],[0.62443]]  ## 테스트용으로 종가 5일치를 역순으로 
scale= MinMaxScaler()
scale.min_, scale.scale_=scaler.min_[4], scaler.scale_[4]   ##  Open열[0],High열[1] ~ Close열은 [4]
data_d = scale.inverse_transform(pred)
print(data_d)

In [ ]:
import numpy as np
print(np.__version__) # 1.16.4

# 결과확인을 위해 소수 출력 옵션 변경
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.1f}".format(x)}) 

from sklearn.preprocessing import MinMaxScaler

t1 =  np.array([
                    [ 1,    1000],
                    [ 5,   10000],
                    [10,  100000],
               ])
t2 =  np.array([
                    [  2,    100],
                    [ 15,  20000],
                    [100, 300000],
               ])

scaler = MinMaxScaler()

scaler.fit(t1)
print(scaler.n_samples_seen_, scaler.data_min_, scaler.data_max_, scaler.feature_range)
# > 3 [1.0 1000.0] [10.0 100000.0] (0, 1)

scaler.partial_fit(t2) # 추가 피팅
print(scaler.n_samples_seen_, scaler.data_min_, scaler.data_max_, scaler.feature_range)
# > 6 [1.0 100.0] [100.0 300000.0] (0, 1)

t2_prinme = scaler.transform(t2)
# array([[0.0, 0.0],
#        [0.1, 0.1],
#        [1.0, 1.0]])
print(t2_prinme)

t2_prime_prime = scaler.inverse_transform(t2_prinme)
#  array([[  2.0,    100.0],
#         [ 15.0,  20000.0],
#         [100.0, 300000.0]])
print(t2_prime_prime)


### https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221614354947&proxyReferer=https:%2F%2Fwww.google.com%2F

## np.reshape

In [ ]:
##  reshape(-1)의 의미  : 가변적이다

x = np.arange(12)
print(x)

print(x.reshape(-1,1))  ## 행,열중 열이  1열이므로 12행이된다
print(x.reshape(-1,2))  ## 열이 2열이므로 6행이 된다

print(x.reshape(1,-1))  ## 행,열중 행이 1행이므로 열이 12열이 된다
print(x.reshape(2,-1))  ## 행,열중 행이 2행이므로 열이 6열이 된다

## 년중 특정요일 날짜 추출하기

In [ ]:
import pandas as pd

def allsundays(year):
    return pd.date_range(start=str(year), end=str(year+1),freq='W-SUN').strftime('%Y-%m-%d').tolist()

allsundays(2020)[:5]  # First 5 Sundays of 2020
# ['01/01/2017', '01/08/2017', '01/15/2017', '01/22/2017', '01/29/2017']


https://rfriend.tistory.com/503

In [ ]:
from datetime import date, timedelta

def allsundays(year):
    d = date(year, 1, 1)                    # January 1st
    d += timedelta(days = 6 - d.weekday())  # First Sunday
    while d.year == year:
        yield d
        d += timedelta(days = 7)

sunday=[]    
for d in allsundays(2020):
    sunday.append(d)

sunday[0]

In [ ]:
## DateBase Table에서 최종날짜 또는 최종날짜 또는 다음날짜 string으로 변환

query = 'select Date from kospi order by Date desc limit 1'
df = pd.read_sql(query, engine)
#df = str(df['Date'])            ##  최종 날짜
#start = df[5:15]
df = pd.to_datetime(df['Date'])
df = df + timedelta(1)          ##  최종날짜 다음날짜
df = str(df)
start = df[4:14]                ## 2020-07-13
start = start.replace('-','')   ## 20200713

start


# 월봉, 주봉 DataFrame 만들기

In [ ]:
##  stock 자료 항목을 월봉, 주봉  DataFrame 으로 변환하기_1
##  kospi 지수 주봉, 월봉 만들기


def day_week_month_data(market='kospi', start_day = '2020-01-01',period ='month'):
    if market=='kospi' or market=='kosdaq':
        df = select_market_period(market,start_day)
    else :
        df = select_stock_period(market,start_day)
        
    df['Date']=pd.to_datetime(df['Date'])
    months = [g for n, g in df.groupby(pd.Grouper(key='Date',freq='M'))]  ##   월별
    weeks = [g for n, g in df.groupby(pd.Grouper(key='Date',freq='W'))]  ##   주별
    columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
    rows = []    

    if period == 'day':
        df=df[['Date','Open', 'High', 'Low','Close', 'Volume']]
        df.columns=columns
        #df = df.set_index(df['date'])
        #return df    
    if period == 'month':
        nick='month'
        period = months
    elif period == 'week':
        nick='week'
        period = weeks
        
    for i in range(len(period)):
        try:
            rows.append(period[i].iloc[-1]['Date'])
            rows.append(period[i].iloc[0]["Open"])
            rows.append(max(period[i]['High']))
            rows.append(min(period[i]['Low']))
            rows.append(period[i].iloc[-1]['Close'])
            rows.append(sum(period[i]['Volume']))
        except:
            pass
    
    print('count:', len(period))
    arr = np.array(rows)
    if  nick == 'week' and len(period) > 177:
        arr1 = arr.reshape(len(period)-1,6)
    else:
        arr1 = arr.reshape(len(period),6)    
    df = pd.DataFrame(data=arr1, columns=columns)
    
    return df

df = day_week_month_data('노바텍',start_day='2020-01-01',period='month')
df

# 장기적분석 close > ma60 > ma120 종목 분석

In [ ]:
from mod1 import *

start=time.time()
print(start)
source_dir = 'd:/stockdata/close_ma120/'
df = all_stock_at('2020-08-03')
df = df['Name']
name = df.to_list()

select_start_a = '2019-01-01'
select_start_b = '2008-01-01'
    
select_query = "select * from market where Name='hrs' and Date >= '2020-02-20' "
df3 = pd.read_sql(select_query, engine)

df3 = df3['Date']
datelist = df3.to_list()    


def search_stock_long_period(select_start):
    print(select_start)
    df2 = pd.DataFrame() 
    for i in name:
        #print(i)
        df=select_stock_period(i,select_start)  ## 종목별 dataframe
        #print(df)
        #pure_df = pure_df.append(df)  ## 전종목 dataframe
        ma(df)

        source = MinMaxScaler()
        data = source.fit_transform(df[['close','ma60','ma120','volume']].values)
        df1 = pd.DataFrame(data)
        df1['name']=i
        df1.columns=['close','ma60','ma120','volume','name']
        df1[['date','code']] = df[['date','code']]
            #print(df1)
        df2 = df2.append(df1)   ## 전종목 close, ma60, ma120, volume 표준화 (MinMaxScaler())

    for i in datelist:
            last_df = df2.loc[df2['date'] == i]  ##  가장최근일자 전종목  (표준화후)
            last_df = last_df.reset_index(drop=True)

            last_close_df = last_df[last_df['close'] < 0.1]   ##  가장최근종가가 최저가 인경우  (표준화 후)
            last_close_df =  last_close_df.sort_values(["close"],ascending=True)

            mask1 = (last_df.ma120 < 0.1) & (last_df.close >last_df.ma60) & (last_df.ma60 > last_df.ma120) ##  boolen 필터링
            last_ma_df = last_df.loc[mask1, :]   ## 가장최근 120일선이 최저인 &  경우  종가 > ma60 > ma120  (표준화 후)
            ast_ma_df =  last_ma_df.sort_values(["ma120"],ascending=True)

            strdate = i.strftime('%Y-%m-%d')

            last_ma_df.to_excel(path_ma+select_start+"_"+strdate+'.xlsx')  ##  표준화 dataframe 중 ma120 < 0.1 and close > ma60 > ma120 (from 2019.01.01)
            last_close_df.to_excel(path_close+select_start+"_"+strdate+'.xlsx')  ##  표준화 dataframe 중 close < 0.1 
   
    programtrend_df = pd.read_sql("select Date from programtrend order by Date desc limit 1", engine)
    programtrend_df = str(programtrend_df['Date'])
    until_date = programtrend_df[10:15]
    until_date = until_date.replace('-','')
    print(source_dir)

    try:
        os.mkdir('d:/kkang')
    except:
        pass
    for filename in glob.glob(os.path.join(source_dir , '*.*')):
        shutil.copy(filename, source_dir+'2021/'+'2021_02/'+until_date+'/')
    print('time:', time.time()-start)
    
search_stock_long_period('2019-01-01')